In [1]:
import pandas as pd
import numpy as np
from googletrans import Translator
import time
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
# from sentence_transformers import SentenceTransformer

In [2]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from rapidfuzz import fuzz
from sklearn.cluster import AgglomerativeClustering

In [3]:
naver_df = pd.read_csv("naver_webtoons.csv")
kakao_df = pd.read_csv("kakao_webtoons_final.csv")
kpage_df = pd.read_csv("kakao_page_webtoons.csv")

In [4]:
naver_df = naver_df.drop(["finish"], axis=1)

In [5]:
naver_df.head()

,titleId,titleName,url,thumbnailUrl,adult,Writer,Painter,Original,synopsis,genre,day
0,839004,만남어플 중독,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/839004...,True,루즌아,루즌아,NaN,20살 비인기 스트리머 초롱. 어플로 만난 정체불명의 남자와의 관계 속에서 주변 사...,"DRAMA, 고자극드라마, 고자극스릴러, 나쁜남자, 드라마, 미스터리, 범죄, 성인...","목, 월"
1,844058,신체,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/844058...,True,엄세윤,정썸머,NaN,"사채업자, 재벌, 꿈에 그리던 첫사랑까지... 모두가 내 몸을 원한다. 대체 왜?","THRILL, 드라마, 범죄, 블랙코미디, 성인웹툰, 스릴러, 이능력, 자극적인, ...",월
2,758037,참교육,https://comic.naver.com/webtoon/list?titleId=7...,https://image-comic.pstatic.net/webtoon/758037...,False,채용택,한가람,NaN,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!\n<부활남>...,"ACTION, 다크히어로, 먼치킨, 블루스트링, 사이다, 사회고발, 액션, 자극적인...",월
3,822657,환생천마,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/822657...,False,JP,부겸,장영훈,"철혈의 맹주, 강호의 절대자 '천하진'. 가문의 수치라 불리는 망나니 '벽리단'의 ...","HISTORICAL, 고인물, 동양, 동양풍판타지, 먼치킨, 무협/사극, 사이다, ...",월
4,844984,20주년 명작 극장,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/844984...,False,"범배, 조석","범배, 조석",NaN,네이버웹툰이 어느덧 20주년을 맞이했다! 그간 어떤 작품들이 네이버웹툰을 빛냈을까?...,"COMIC, 개그, 열혈병맛개그","금, 월, 일, 화"


In [6]:
kakao_df.head()

,웹툰ID,제목,url,thumbnailurl,카피문구,성인여부,요일,시놉시스,키워드리스트,글작가,그림작가,원작가
0,198,오무라이스 잼잼,https://webtoon.kakao.com/content/오무라이스-잼잼/198,https://kr-a.kakaopagecdn.com/P/C/198/c1/2x/dc...,다이어트 할 때 가장 위험한 만화\n추억과 음식과 이야기들,False,화,'음식'과 '가족'의 이야기를 담은 에세이 만화.\n\n몇 개라도 먹을 수 있을 것...,"공감되는, 식욕을 자극하는, 코믹/일상, 지식정보, COMIC_EVERYDAY_LIFE",조경규,조경규,NaN
1,760,딩스뚱스,https://webtoon.kakao.com/content/딩스뚱스/760,https://kr-a.kakaopagecdn.com/P/C/760/c1/2x/29...,"토종 한국인 딩스와 뚱스,\n타지에서 맨땅에 헤딩하기!",False,화,"토종 한국인 딩스와 뚱스, 타지에서 맨땅에 헤딩하기!","귀여운, 따뜻한, 코믹/일상, 에피소드물, COMIC_EVERYDAY_LIFE",딩스,딩스,NaN
2,792,블랙 베히모스,https://webtoon.kakao.com/content/블랙-베히모스/792,https://kr-a.kakaopagecdn.com/P/C/792/c1/2x/48...,탈리스만이 되기 위한 시험에 \n도전하는 주인공의 힘겨운 전투!,False,수,"태초에 세 명의 신이 있었다.\n땅의 베히모스, 바다의 레비야탄, 하늘의 즈가.\n...","몰입되는, 압도되는, 학원/판타지, 성장물, SCHOOL_ACTION_FANTASY",케이지콘,케이지콘,NaN
3,823,레드스톰 - 왕의 귀환,https://webtoon.kakao.com/content/레드스톰---왕의-귀환...,https://kr-a.kakaopagecdn.com/P/C/823/c1/2x/fc...,"4년 만에 돌아온 그로우, 왕의 귀환\n사막을 넘어 다시 시작되는 전쟁",False,토,피에 젖은 사막.\n붉은 사막을 용납할 수 없던 한 소년.\n\n소년은 사막의 무신...,"역동적인, 몰입되는, 액션/무협, 성공성장물, ACTION_WUXIA",노경찬,암현,NaN
4,1078,유부녀의 탄생,https://webtoon.kakao.com/content/유부녀의-탄생/1078,https://kr-a.kakaopagecdn.com/P/C/1078/c1/2x/0...,누구도 알려주지 않은 결혼식 이후\n삶에 대한 리얼리티 카툰!,False,월,누구도 알려주지 않은 \n결혼식 이후 삶에 대한 리얼리티 카툰!,"공감되는, 귀여운, 코믹/일상, 에피소드물, COMIC_EVERYDAY_LIFE",김환타,김환타,NaN


In [7]:
kpage_df.head()

,series_id,title,url,category,thumbnailUrl,description,age_limit_detail,weekday,writer,illustrator,original_author
0,46609100,은동스+옹동스,https://page.kakao.com/content/46609100,드라마,//page-images.kakaoentcdn.com/download/resourc...,오빠와 이별했지만 귀엽고 순수한 작은 고양이 은동은 오늘도 엄마와 함께 밝고 씩씩하...,All,금,스노우캣,스노우캣,NaN
1,47196143,달빛조각사,https://page.kakao.com/content/47196143,"판타지, 게임, 전문직, 퓨전, 전쟁, 생존, 깽판, 먼치킨, 시스템, 천재, 노력...",//page-images.kakaoentcdn.com/download/resourc...,"아르바이트를 전전하며 어렵게 살던 소년가장 이현, 그가 ‘위드’가 되어 가상현실게임...",All,화,"김준형, 이도경","김태형, 박정열, 신C",남희성
2,47497773,트레이스 - 신들의 시선,https://page.kakao.com/content/47497773,"액션, 퓨전, 먼치킨, 초능력",//page-images.kakaoentcdn.com/download/resourc...,어느날 갑자기 세상에 나타나 인간들을 공격하기 시작한 정체불명의 존재 트러블.\n트...,All,화,네스티캣,네스티캣,NaN
3,47740585,나의 그녀는 구미호,https://page.kakao.com/content/47740585,판타지,//page-images.kakaoentcdn.com/download/resourc...,어느날 나타난 사람을 빨아 먹는 나무요괴 미녀..\n그리고 양기가 충만한 주인공 봉...,Fifteen,수,프리,서락,NaN
4,47936014,신인왕좌,https://page.kakao.com/content/47936014,"판타지, 천재, 검사, 마법사, 격투가, 용병, 정령술사, 용사, 힐러",//page-images.kakaoentcdn.com/download/resourc...,유명 판타지 소설을 만화화 한 작품!\n\n마족이 강세하고 인류는 멸종 위기에 직면...,All,토,Sariel,Sariel,NaN


In [8]:
kakao = kakao_df.rename(columns={
    "웹툰ID": "titleId",
    "제목": "titleName",
    "thumbnailurl": "thumbnailUrl",
    "url": "Url",
    "성인여부": "is_adult",
    "시놉시스": "synopsis",
    "글작가": "Writer",
    "그림작가": "Painter",
    "원작가": "Original",
    "키워드리스트": "genre",   # or 별도 만든 장르+키워드 컬럼
    "요일": "day",
})[["titleId","titleName","Url","thumbnailUrl","is_adult",
    "Writer","Painter","Original","synopsis","genre","day"]]

kakao

,titleId,titleName,Url,thumbnailUrl,is_adult,Writer,Painter,Original,synopsis,genre,day
0,198,오무라이스 잼잼,https://webtoon.kakao.com/content/오무라이스-잼잼/198,https://kr-a.kakaopagecdn.com/P/C/198/c1/2x/dc...,False,조경규,조경규,NaN,'음식'과 '가족'의 이야기를 담은 에세이 만화.\n\n몇 개라도 먹을 수 있을 것...,"공감되는, 식욕을 자극하는, 코믹/일상, 지식정보, COMIC_EVERYDAY_LIFE",화
1,760,딩스뚱스,https://webtoon.kakao.com/content/딩스뚱스/760,https://kr-a.kakaopagecdn.com/P/C/760/c1/2x/29...,False,딩스,딩스,NaN,"토종 한국인 딩스와 뚱스, 타지에서 맨땅에 헤딩하기!","귀여운, 따뜻한, 코믹/일상, 에피소드물, COMIC_EVERYDAY_LIFE",화
2,792,블랙 베히모스,https://webtoon.kakao.com/content/블랙-베히모스/792,https://kr-a.kakaopagecdn.com/P/C/792/c1/2x/48...,False,케이지콘,케이지콘,NaN,"태초에 세 명의 신이 있었다.\n땅의 베히모스, 바다의 레비야탄, 하늘의 즈가.\n...","몰입되는, 압도되는, 학원/판타지, 성장물, SCHOOL_ACTION_FANTASY",수
3,823,레드스톰 - 왕의 귀환,https://webtoon.kakao.com/content/레드스톰---왕의-귀환...,https://kr-a.kakaopagecdn.com/P/C/823/c1/2x/fc...,False,노경찬,암현,NaN,피에 젖은 사막.\n붉은 사막을 용납할 수 없던 한 소년.\n\n소년은 사막의 무신...,"역동적인, 몰입되는, 액션/무협, 성공성장물, ACTION_WUXIA",토
4,1078,유부녀의 탄생,https://webtoon.kakao.com/content/유부녀의-탄생/1078,https://kr-a.kakaopagecdn.com/P/C/1078/c1/2x/0...,False,김환타,김환타,NaN,누구도 알려주지 않은 \n결혼식 이후 삶에 대한 리얼리티 카툰!,"공감되는, 귀여운, 코믹/일상, 에피소드물, COMIC_EVERYDAY_LIFE",월
...,...,...,...,...,...,...,...,...,...,...,...
1021,4696,뱀파이어 로드로 사는 법,https://webtoon.kakao.com/content/뱀파이어-로드로-사는-...,https://kr-a.kakaopagecdn.com/P/C/4696/c1/2x/b...,False,목수,스타우트,김형준(프로즌),반년간의 도주 생활 끝에 결국 성기사의 명성을 위한 제물로 살해당한 뱀파이어 유진....,"성장하는, 몰입되는, 학원/판타지, 회귀물, SCHOOL_ACTION_FANTASY",금
1022,4697,남주들과 외딴섬에 갇혀버렸다,https://webtoon.kakao.com/content/남주들과-외딴섬에-갇혀...,https://kr-a.kakaopagecdn.com/P/C/4697/c1/2x/9...,False,몽당,노논,김지아,소설 속 남주들과 외딴섬에 갇혀버렸다.\n그것도 19금 피폐 소설에 나오는 남주들과...,"설레는, 역동적인, 로맨스 판타지, 빙의물, ROMANCE_FANTASY",일
1023,4698,회귀한 악녀는 순결서약 합니다,https://webtoon.kakao.com/content/회귀한-악녀는-순결서약...,https://kr-a.kakaopagecdn.com/P/C/4698/c1/2x/6...,False,MOL,한연,한처음,"사랑하는 남자에게 간이고 쓸개고 다 바쳐 그를 황제로 만든 후작가 영애, 리디아 에...","발랄한, 유머러스한, 로맨스 판타지, 회귀물, ROMANCE_FANTASY",월
1024,4699,드래곤을 유괴하다,https://webtoon.kakao.com/content/드래곤을-유괴하다/4699,https://kr-a.kakaopagecdn.com/P/C/4699/c1/2x/f...,False,최진형,주황새,유주,최강의 힘을 가진 회귀자 유지태.\n그만한 힘을 가지고 거듭 회귀를 했음에도 지구의...,"역동적인, 귀여운, 판타지 드라마, 회귀물, FANTASY_DRAMA",목


In [10]:
kpage_df

,series_id,title,url,category,thumbnailUrl,description,age_limit_detail,weekday,writer,illustrator,original_author
0,46609100,은동스+옹동스,https://page.kakao.com/content/46609100,드라마,//page-images.kakaoentcdn.com/download/resourc...,오빠와 이별했지만 귀엽고 순수한 작은 고양이 은동은 오늘도 엄마와 함께 밝고 씩씩하...,All,금,스노우캣,스노우캣,NaN
1,47196143,달빛조각사,https://page.kakao.com/content/47196143,"판타지, 게임, 전문직, 퓨전, 전쟁, 생존, 깽판, 먼치킨, 시스템, 천재, 노력...",//page-images.kakaoentcdn.com/download/resourc...,"아르바이트를 전전하며 어렵게 살던 소년가장 이현, 그가 ‘위드’가 되어 가상현실게임...",All,화,"김준형, 이도경","김태형, 박정열, 신C",남희성
2,47497773,트레이스 - 신들의 시선,https://page.kakao.com/content/47497773,"액션, 퓨전, 먼치킨, 초능력",//page-images.kakaoentcdn.com/download/resourc...,어느날 갑자기 세상에 나타나 인간들을 공격하기 시작한 정체불명의 존재 트러블.\n트...,All,화,네스티캣,네스티캣,NaN
3,47740585,나의 그녀는 구미호,https://page.kakao.com/content/47740585,판타지,//page-images.kakaoentcdn.com/download/resourc...,어느날 나타난 사람을 빨아 먹는 나무요괴 미녀..\n그리고 양기가 충만한 주인공 봉...,Fifteen,수,프리,서락,NaN
4,47936014,신인왕좌,https://page.kakao.com/content/47936014,"판타지, 천재, 검사, 마법사, 격투가, 용병, 정령술사, 용사, 힐러",//page-images.kakaoentcdn.com/download/resourc...,유명 판타지 소설을 만화화 한 작품!\n\n마족이 강세하고 인류는 멸종 위기에 직면...,All,토,Sariel,Sariel,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1492,68144406,이리초파랑,https://page.kakao.com/content/68144406,"무협, 정통, 생존, 복수, 무림, 정파, 협객",//page-images.kakaoentcdn.com/download/resourc...,"""보국구대문(保國九大門)의 영웅들이여---!!”\n\n멸국의 위기에 의기로 일어선\...",Fifteen,금,사마달,사마달,NaN
1493,68148725,붉은 달밤의 끝자락에서,https://page.kakao.com/content/68148725,"BL, 동양풍, 운명, 다정, 사랑꾼, 절대강, 집착, 소심, 아방, 소유욕, 운명...",//page-images.kakaoentcdn.com/download/resourc...,"저주받은 아이라 손가락질 받으며 외롭게 살아온 자운.\n\n어느 날, 괴질이 온 마...",Fifteen,토,오공시,오공시,NaN
1494,68148783,붉은 달밤의 끝자락에서 [19세 완전판],https://page.kakao.com/content/68148783,"BL, 동양풍, 운명, 다정, 사랑꾼, 절대강, 집착, 소심, 아방, 소유욕, 질투...",//page-images.kakaoentcdn.com/download/resourc...,"저주받은 아이라 손가락질 받으며 외롭게 살아온 자운.\n\n어느 날, 괴질이 온 마...",Nineteen,토,오공시,오공시,NaN
1495,68187457,드래곤을 유괴하다,https://page.kakao.com/content/68187457,"판타지, 퓨전, 회귀, 먼치킨, 육아, 성장, 이종족",//page-images.kakaoentcdn.com/download/resourc...,최강의 힘을 가진 회귀자 유지태.\n그만한 힘을 가지고 거듭 회귀를 했음에도 지구의...,Fifteen,목,최진형,주황새,유주


In [11]:
kpage = kpage_df.rename(columns={
    "series_id": "titleId",
    "title": "titleName",
    "url": "Url",
    "thumbnailUrl": "thumbnailUrl",
    "description": "synopsis",
    "writer": "Writer",
    "illustrator": "Painter",
    "original_author": "Original",
    "category": "genre",
    "weekday": "day",
})

adult_map = {"All": False, "Fifteen": False, "Nineteen": True}
kpage["is_adult"] = kpage["age_limit_detail"].map(adult_map).fillna(False)

kpage = kpage[["titleId","titleName","Url","thumbnailUrl","is_adult",
             "Writer","Painter","Original","synopsis","genre","day"]]

kpage

,titleId,titleName,Url,thumbnailUrl,is_adult,Writer,Painter,Original,synopsis,genre,day
0,46609100,은동스+옹동스,https://page.kakao.com/content/46609100,//page-images.kakaoentcdn.com/download/resourc...,False,스노우캣,스노우캣,NaN,오빠와 이별했지만 귀엽고 순수한 작은 고양이 은동은 오늘도 엄마와 함께 밝고 씩씩하...,드라마,금
1,47196143,달빛조각사,https://page.kakao.com/content/47196143,//page-images.kakaoentcdn.com/download/resourc...,False,"김준형, 이도경","김태형, 박정열, 신C",남희성,"아르바이트를 전전하며 어렵게 살던 소년가장 이현, 그가 ‘위드’가 되어 가상현실게임...","판타지, 게임, 전문직, 퓨전, 전쟁, 생존, 깽판, 먼치킨, 시스템, 천재, 노력...",화
2,47497773,트레이스 - 신들의 시선,https://page.kakao.com/content/47497773,//page-images.kakaoentcdn.com/download/resourc...,False,네스티캣,네스티캣,NaN,어느날 갑자기 세상에 나타나 인간들을 공격하기 시작한 정체불명의 존재 트러블.\n트...,"액션, 퓨전, 먼치킨, 초능력",화
3,47740585,나의 그녀는 구미호,https://page.kakao.com/content/47740585,//page-images.kakaoentcdn.com/download/resourc...,False,프리,서락,NaN,어느날 나타난 사람을 빨아 먹는 나무요괴 미녀..\n그리고 양기가 충만한 주인공 봉...,판타지,수
4,47936014,신인왕좌,https://page.kakao.com/content/47936014,//page-images.kakaoentcdn.com/download/resourc...,False,Sariel,Sariel,NaN,유명 판타지 소설을 만화화 한 작품!\n\n마족이 강세하고 인류는 멸종 위기에 직면...,"판타지, 천재, 검사, 마법사, 격투가, 용병, 정령술사, 용사, 힐러",토
...,...,...,...,...,...,...,...,...,...,...,...
1492,68144406,이리초파랑,https://page.kakao.com/content/68144406,//page-images.kakaoentcdn.com/download/resourc...,False,사마달,사마달,NaN,"""보국구대문(保國九大門)의 영웅들이여---!!”\n\n멸국의 위기에 의기로 일어선\...","무협, 정통, 생존, 복수, 무림, 정파, 협객",금
1493,68148725,붉은 달밤의 끝자락에서,https://page.kakao.com/content/68148725,//page-images.kakaoentcdn.com/download/resourc...,False,오공시,오공시,NaN,"저주받은 아이라 손가락질 받으며 외롭게 살아온 자운.\n\n어느 날, 괴질이 온 마...","BL, 동양풍, 운명, 다정, 사랑꾼, 절대강, 집착, 소심, 아방, 소유욕, 운명...",토
1494,68148783,붉은 달밤의 끝자락에서 [19세 완전판],https://page.kakao.com/content/68148783,//page-images.kakaoentcdn.com/download/resourc...,True,오공시,오공시,NaN,"저주받은 아이라 손가락질 받으며 외롭게 살아온 자운.\n\n어느 날, 괴질이 온 마...","BL, 동양풍, 운명, 다정, 사랑꾼, 절대강, 집착, 소심, 아방, 소유욕, 질투...",토
1495,68187457,드래곤을 유괴하다,https://page.kakao.com/content/68187457,//page-images.kakaoentcdn.com/download/resourc...,False,최진형,주황새,유주,최강의 힘을 가진 회귀자 유지태.\n그만한 힘을 가지고 거듭 회귀를 했음에도 지구의...,"판타지, 퓨전, 회귀, 먼치킨, 육아, 성장, 이종족",목


In [12]:
naver = naver_df.rename(columns={
    "url": "Url",
    "adult": "is_adult",
})

naver = naver[["titleId","titleName","Url","thumbnailUrl","is_adult",
                  "Writer","Painter","Original","synopsis","genre","day"]]

naver

,titleId,titleName,Url,thumbnailUrl,is_adult,Writer,Painter,Original,synopsis,genre,day
0,839004,만남어플 중독,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/839004...,True,루즌아,루즌아,NaN,20살 비인기 스트리머 초롱. 어플로 만난 정체불명의 남자와의 관계 속에서 주변 사...,"DRAMA, 고자극드라마, 고자극스릴러, 나쁜남자, 드라마, 미스터리, 범죄, 성인...","목, 월"
1,844058,신체,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/844058...,True,엄세윤,정썸머,NaN,"사채업자, 재벌, 꿈에 그리던 첫사랑까지... 모두가 내 몸을 원한다. 대체 왜?","THRILL, 드라마, 범죄, 블랙코미디, 성인웹툰, 스릴러, 이능력, 자극적인, ...",월
2,758037,참교육,https://comic.naver.com/webtoon/list?titleId=7...,https://image-comic.pstatic.net/webtoon/758037...,False,채용택,한가람,NaN,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!\n<부활남>...,"ACTION, 다크히어로, 먼치킨, 블루스트링, 사이다, 사회고발, 액션, 자극적인...",월
3,822657,환생천마,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/822657...,False,JP,부겸,장영훈,"철혈의 맹주, 강호의 절대자 '천하진'. 가문의 수치라 불리는 망나니 '벽리단'의 ...","HISTORICAL, 고인물, 동양, 동양풍판타지, 먼치킨, 무협/사극, 사이다, ...",월
4,844984,20주년 명작 극장,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/844984...,False,"범배, 조석","범배, 조석",NaN,네이버웹툰이 어느덧 20주년을 맞이했다! 그간 어떤 작품들이 네이버웹툰을 빛냈을까?...,"COMIC, 개그, 열혈병맛개그","금, 월, 일, 화"
...,...,...,...,...,...,...,...,...,...,...,...
761,800101,헬스던전,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/800101...,False,도베도베,채종,NaN,"인류 최강의 헬스남 한솔은 데드리프트 도중 정신을 잃었는데, 눈을 떠보니 이세계에 ...","ACTION, 4차원, 먼치킨, 사이다, 액션, 이세계",일
762,836948,우리의 공백,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/836948...,False,조예빈,조예빈,NaN,누군가 떠나는것이 두려워 회피형으로 자란 '희진'\n자신과 정반대의 모습이 된 소꿉...,"PURE, 감성적인, 구원서사, 로맨스, 재회, 청춘로맨스, 학원물",일
763,837417,요괴삼월,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/837417...,False,이윤희,이윤희,NaN,전 태권도 국가대표 선수 이은비. 슬럼프에 빠져 운동을 그만두고 그런대로 살아가던 ...,"FANTASY, 동양풍로맨스, 러브코미디, 로맨틱코미디, 순진남, 판타지, 현대, ...",일
764,797155,킬링킬러,https://comic.naver.com/webtoon/list?titleId=7...,https://image-comic.pstatic.net/webtoon/797155...,False,아백,아백,NaN,"10년 전, 살인마에게 부모님을 잃은 주인공 이시우. 마침내 부모님을 죽인 살인마 ...","ACTION, 범죄, 복수극, 시리어스, 액션, 이능력, 이능력배틀물, 자극적인, ...",일


In [13]:
# 네이버
naver["provider"] = "NAVER"

# 카카오 웹툰
kakao["provider"] = "KAKAO"

# 카카오페이지
kpage["provider"] = "KAKAOPAGE"

In [14]:
all_webtoons = pd.concat([naver, kakao, kpage], ignore_index=True)
all_webtoons.head()

,titleId,titleName,Url,thumbnailUrl,is_adult,Writer,Painter,Original,synopsis,genre,day,provider
0,839004,만남어플 중독,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/839004...,True,루즌아,루즌아,NaN,20살 비인기 스트리머 초롱. 어플로 만난 정체불명의 남자와의 관계 속에서 주변 사...,"DRAMA, 고자극드라마, 고자극스릴러, 나쁜남자, 드라마, 미스터리, 범죄, 성인...","목, 월",NAVER
1,844058,신체,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/844058...,True,엄세윤,정썸머,NaN,"사채업자, 재벌, 꿈에 그리던 첫사랑까지... 모두가 내 몸을 원한다. 대체 왜?","THRILL, 드라마, 범죄, 블랙코미디, 성인웹툰, 스릴러, 이능력, 자극적인, ...",월,NAVER
2,758037,참교육,https://comic.naver.com/webtoon/list?titleId=7...,https://image-comic.pstatic.net/webtoon/758037...,False,채용택,한가람,NaN,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!\n<부활남>...,"ACTION, 다크히어로, 먼치킨, 블루스트링, 사이다, 사회고발, 액션, 자극적인...",월,NAVER
3,822657,환생천마,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/822657...,False,JP,부겸,장영훈,"철혈의 맹주, 강호의 절대자 '천하진'. 가문의 수치라 불리는 망나니 '벽리단'의 ...","HISTORICAL, 고인물, 동양, 동양풍판타지, 먼치킨, 무협/사극, 사이다, ...",월,NAVER
4,844984,20주년 명작 극장,https://comic.naver.com/webtoon/list?titleId=8...,https://image-comic.pstatic.net/webtoon/844984...,False,"범배, 조석","범배, 조석",NaN,네이버웹툰이 어느덧 20주년을 맞이했다! 그간 어떤 작품들이 네이버웹툰을 빛냈을까?...,"COMIC, 개그, 열혈병맛개그","금, 월, 일, 화",NAVER


In [15]:
all_webtoons.to_csv('all_webtoons.csv', index=False)

In [13]:
# 제목별로 몇 개의 서로 다른 provider가 있는지 집계
g = (
    all_webtoons
    .groupby('titleName')['provider']
    .nunique()
    .reset_index(name='n_providers')
)

# 2개 이상 플랫폼에 있는 제목만 필터
multi_platform_titles = g[g['n_providers'] > 1]['titleName']

# 실제 행들 뽑기
df_multi = (
    all_webtoons[all_webtoons['titleName'].isin(multi_platform_titles)]
    .sort_values(['titleName', 'provider'])
)

df_multi

,titleId,titleName,Url,thumbnailUrl,is_adult,Writer,Painter,Original,synopsis,genre,day,provider
1325,4117,0.0000001%의 마왕님,https://webtoon.kakao.com/content/0.0000001%의-...,https://kr-a.kakaopagecdn.com/P/C/4117/c1/2x/9...,False,유월,솜사탕,파란영,지상계를 멸망 시키라는 명을 받은 72인의 마족\n\n대마왕 아스트레아의 시험을 통...,"역동적인, 긴장감 있는, 액션/무협, 모험물, ACTION_WUXIA",월,KAKAO
2611,63857841,0.0000001%의 마왕님,https://page.kakao.com/content/63857841,//dn-img-page.kakao.com/download/resource?kid=...,False,"유월, 화기연",솜사탕,파란영,지상계를 멸망 시키라는 명을 받은 72인의 마족\n\n대마왕 아스트레아의 시험을 통...,"판타지, 마왕, 마계, 라이트노벨, 영웅/신화, 이능력",월,KAKAOPAGE
1077,3476,0살부터 슈퍼스타,https://webtoon.kakao.com/content/0살부터-슈퍼스타/3476,https://kr-a.kakaopagecdn.com/P/C/3476/c1/2x/5...,False,캐롯스튜디오,캐롯스튜디오,북그곰,"엘프, 요정, 슬라임, 선물상자, 마왕 등\n수천 번의 환생에 이어 몇 번째인지도 ...","따뜻한, 감동적인, 판타지 드라마, 성공성장물, FANTASY_DRAMA",일,KAKAO
2338,61488489,0살부터 슈퍼스타,https://page.kakao.com/content/61488489,//dn-img-page.kakao.com/download/resource?kid=...,False,NaN,캐롯스튜디오,북그곰,"엘프, 요정, 슬라임, 선물상자, 마왕 등\n수천 번의 환생에 이어 몇 번째인지도 ...","드라마, 일상물, 판타지물, 현대물, 성장물, 일상, 전생/환생, 환생물, 배우, ...",일,KAKAOPAGE
1671,4575,101번째 고백,https://webtoon.kakao.com/content/101번째-고백/4575,https://kr-a.kakaopagecdn.com/P/C/4575/c1/2x/8...,False,팀 게코,팀 라일락,현솔,“헤르윈! 너를 좋아해! 나랑 사귀자!”\n\n13년 동안 짝사랑하는 상대에게 98...,"몰입되는, 가슴 먹먹한, 로맨스 판타지, 연애물, ROMANCE_FANTASY",금,KAKAO
...,...,...,...,...,...,...,...,...,...,...,...,...
2940,66066222,힐링물 속 피폐 엑스트라가 되었다,https://page.kakao.com/content/66066222,//dn-img-page.kakao.com/download/resource?kid=...,False,서시내,김표고,플로우,어느 날 힐링 육아물 속 엑스트라에 빙의했다.\n사랑스러운 황녀와 모두가 행복하게 ...,"로판, 로맨스판타지, 육아물, 츤데레남, 능력녀, 다정녀, 성장물, 상처녀, 까칠남...",금,KAKAOPAGE
946,2990,힐통령 - 태양의 사제,https://webtoon.kakao.com/content/힐통령---태양의-사제...,https://kr-a.kakaopagecdn.com/P/C/2990/c1/2x/1...,False,"레드고, 용궁뎅이","방승호(STUDIO GRIFFIN), 손호원",제리엠,"""저런 퀘스트를 하는 건 착해서가 아니라 그냥 호구인 거야, 호구.""\n\n등 뒤에...","통쾌한, 압도되는, 학원/판타지, 레벨업물, SCHOOL_ACTION_FANTASY",화,KAKAO
2096,58710388,힐통령 - 태양의 사제,https://page.kakao.com/content/58710388,//dn-img-page.kakao.com/download/resource?kid=...,False,"레드고, 용궁뎅이","방승호(STUDIO GRIFFIN), 손호원, 윤지영",제리엠,"[매주 화요일 업데이트]\n\n\n""저런 퀘스트를 하는 건 착해서가 아니라 그냥 호...","판타지, 게임, 시스템, 성장, 레이드, 힐러, 플레이어, 마법사, 만능",화,KAKAOPAGE
1726,4634,힘을 숨겨도 공녀가 제일 강함,https://webtoon.kakao.com/content/힘을-숨겨도-공녀가-제...,https://kr-a.kakaopagecdn.com/P/C/4634/c1/2x/a...,False,야옹짹,구마곰,설윤영,"제국의 수호자, ‘영웅’.\n\n강산이 수없이 변하는 긴긴 시간, \n영웅은 늘 황...","통쾌한, 웃기는, 로맨스 판타지, 빙의물, ROMANCE_FANTASY",금,KAKAO


In [15]:
g = (
    all_webtoons
    .groupby('titleName')['provider']
    .agg(
        n_providers='nunique',      # 몇 개 플랫폼에 있는지
        providers_list=lambda x: set(x)  # 실제 플랫폼 집합
    )
    .reset_index()
)

only_kakao_titles = g[
    (g['n_providers'] == 1) &
    (g['providers_list'] == {'KAKAO'})
]['titleName']

df_kakao_only = all_webtoons[all_webtoons['titleName'].isin(only_kakao_titles)].copy()
df_kakao_only

,titleId,titleName,Url,thumbnailUrl,is_adult,Writer,Painter,Original,synopsis,genre,day,provider
780,1768,타원을 그리는 법,https://webtoon.kakao.com/content/타원을-그리는-법/1768,https://kr-a.kakaopagecdn.com/P/C/1768/c1/2x/c...,False,섬멍,섬멍,NaN,오합지졸들과 함께\n회사의 비밀 프로젝트를 밝혀내고\n연인을 과거에서 구출하라!\n...,"기발한, 궁금하게 하는, 로맨스, GL, ROMANCE",월,KAKAO
853,2518,성좌들이 나만 좋아해,https://webtoon.kakao.com/content/성좌들이-나만-좋아해/...,https://kr-a.kakaopagecdn.com/P/C/2518/c1/2x/b...,False,NaN,슈퍼코믹스스튜디오,코기베어,미스테리한 탑이 생겨나면서 세상이 멸망했다.\n탑은 오로지 자신의 주위에만 빛을 내...,"역동적인, 몰입되는, 학원/판타지, 레벨업물, SCHOOL_ACTION_FANTASY",수,KAKAO
872,2651,만렙부터 레벨업,https://webtoon.kakao.com/content/만렙부터-레벨업/2651,https://kr-a.kakaopagecdn.com/P/C/2651/c1/2x/3...,False,스튜디오 시그마,스튜디오 시그마,해무극,"외공만렙, 내공만렙\n강호무림의 명실상부한 천하제일인 차신현.\n\n무공의 끝에 도...","압도되는, 역동적인, 액션/무협, 레벨업물, ACTION_WUXIA",토,KAKAO
880,2694,비선실세 레이디,https://webtoon.kakao.com/content/비선실세-레이디/2694,https://kr-a.kakaopagecdn.com/P/C/2694/c1/2x/9...,False,멍이,청해,예훤,역하렘 소설 속 여주인공의 하나뿐인 동생으로 빙의했다.\n게다가 한 번 크케 앓다 ...,"귀여운, 따뜻한, 로맨스 판타지, 빙의물, ROMANCE_FANTASY",월,KAKAO
896,2782,호위무사,https://webtoon.kakao.com/content/호위무사/2782,https://kr-a.kakaopagecdn.com/P/C/2782/c1/2x/d...,False,w2s,근비,초우,강호 제일 용부의 권좌를 차지하기 위한 숨 가쁜 암투.\n그 위에 엇갈린 두 남녀의...,"역동적인, 압도되는, 액션/무협, 암투물, ACTION_WUXIA",금,KAKAO
908,2833,이 헌터 실화냐,https://webtoon.kakao.com/content/이-헌터-실화냐/2833,https://kr-a.kakaopagecdn.com/P/C/2833/c1/2x/c...,False,스리라챠,스리라챠,파르나르,몬스터'라는 존재가 등장하여 인류를 위협하는 사회.\n\n 아무런 예고 없이 등장한...,"역동적인, 몰입되는, 액션/무협, 레벨업물, ACTION_WUXIA",일,KAKAO
920,2897,아씨의 법정,https://webtoon.kakao.com/content/아씨의-법정/2897,https://kr-a.kakaopagecdn.com/P/C/2897/c1/2x/9...,False,이오삼,반숙,NaN,"세손빈으로 설레는 나날을 보내던 때,\n아버지가 역모의 누명을 쓰고 집안이 몰락했다...","설레는, 감동적인, 로맨스, 남장여자물, ROMANCE",수,KAKAO
924,2905,"영웅, 회귀하다",https://webtoon.kakao.com/content/영웅-회귀하다/2905,https://kr-a.kakaopagecdn.com/P/C/2905/c1/2x/b...,False,앤트스튜디오,앤트스튜디오,흑아인,"인류 최강의 영웅, 김성인. \n모든 것을 내던지며 분투했지만\n결국…… 세상의 멸...","몰입되는, 역동적인, 액션/무협, 회귀물, ACTION_WUXIA",일,KAKAO
926,2922,검빨로 레벨업,https://webtoon.kakao.com/content/검빨로-레벨업/2922,https://kr-a.kakaopagecdn.com/P/C/2922/c1/2x/6...,False,기성욱,BOL×BAE,메켄로,주인공 레온은 오직 노력과 근성으로 같은 왕립아카데미의 정점에 서있는 라이벌 리안을...,"몰입되는, 역동적인, 액션/무협, 성공성장물, ACTION_WUXIA",금,KAKAO
935,2955,검은 달이 뜨면,https://webtoon.kakao.com/content/검은-달이-뜨면/2955,https://kr-a.kakaopagecdn.com/P/C/2955/c1/2x/a...,False,우쥬,보령,유미미,평생을 가문과 가주에게 충성했지만 돌아온 것은 첩자라는 누명과 가족의 죽음 뿐이었다...,"몰입되는, 설레는, 로맨스 판타지, 회귀물, ROMANCE_FANTASY",일,KAKAO


In [29]:
all_webtoons[all_webtoons["is_adult"]==True]

,titleId,titleName,thumbnailUrl,is_adult,Writer,Painter,Original,synopsis,genre,day,provider
0,839004,만남어플 중독,https://image-comic.pstatic.net/webtoon/839004...,True,루즌아,루즌아,NaN,20살 비인기 스트리머 초롱. 어플로 만난 정체불명의 남자와의 관계 속에서 주변 사...,"DRAMA, 고자극드라마, 고자극스릴러, 나쁜남자, 드라마, 미스터리, 범죄, 성인...","목, 월",NAVER
1,844058,신체,https://image-comic.pstatic.net/webtoon/844058...,True,엄세윤,정썸머,NaN,"사채업자, 재벌, 꿈에 그리던 첫사랑까지... 모두가 내 몸을 원한다. 대체 왜?","THRILL, 드라마, 범죄, 블랙코미디, 성인웹툰, 스릴러, 이능력, 자극적인, ...",월,NAVER
5,817859,왕과의 야행,https://image-comic.pstatic.net/webtoon/817859...,True,보리,보리,NaN,젊은 왕과 아름다운 여인이 발을 사이에 두고 앉아있다.\n가느다란 흰 손가락으로 책...,"PURE, 고자극로맨스, 공식미녀, 궁중로맨스, 나쁜남자, 동양, 동양풍로맨스, 로...",월,NAVER
14,796867,버림받은 왕녀의 은밀한 침실,https://image-comic.pstatic.net/webtoon/796867...,True,혜니,혜니,성혜림,한때는 추앙받았지만 지금은 비참하게 버림받은 왕녀 르웰린. 아무도 보호해 주지 않는...,"PURE, 계략남, 고자극로맨스, 군림녀, 궁중로맨스, 까칠남, 능력녀, 다정남, ...",월,NAVER
26,826348,잘 헤어지는 방법,https://image-comic.pstatic.net/webtoon/826348...,True,강문송,1172,NaN,"외로웠던 세진의 인생에 나타난 완벽한 남자, 도영.\n그러나 행복도 잠시,\n갈수록...","THRILL, 계략남, 계략녀, 고자극스릴러, 두뇌싸움, 드라마, 레드스트링, 범죄...",월,NAVER
...,...,...,...,...,...,...,...,...,...,...,...
3231,68035882,구원자의 퍼스트 레이디 [19세 완전판],//dn-img-page.kakao.com/download/resource?kid=...,True,뭬진,소스,리사벨,"망해버린 결혼 생활에서 탈출하기 위해, 인생을 바꾸려고 했다.\n그런데 문제가 조금...","로판, 로맨스판타지, 타임슬립, 재회물, 차원이동물, 복수물, 짝사랑, 운명, 능력...",화,KAKAOPAGE
3232,68035917,저택 영춘화:이혼 후 첫날밤 [19세 완전판],//dn-img-page.kakao.com/download/resource?kid=...,True,모모,이상미,NaN,"\t\n1940년, 혼돈의 시대\n한 남자가 가슴 깊이 사랑한 아내를 위하는 마음만...","로맨스, 현대로맨스, 오해물, 원나잇, 옴니버스, 우연한만남, 운명, 소유욕/독점욕...",목,KAKAOPAGE
3236,68049129,강철의 열제 [19세 완전판],//dn-img-page.kakao.com/download/resource?kid=...,True,수리,신인성,가우리,"이 땅에 가장 영광된 이름, 가우리.\n\n그 이름을 지킨 자.\n사람들은 그를 강...","판타지, 전쟁, 생존, 먼치킨, 영지, 성장, 천재, 노력, 차원이동",월,KAKAOPAGE
3239,68067965,시녀의 유혹,//dn-img-page.kakao.com/download/resource?kid=...,True,밀롱,쥐곰이,쥐똥새똥,3개월의 시한부 선고를 받은 시녀 '이포 벨'은\n그동안 짝사랑해 왔던 후작가의 주...,"로판, 로맨스판타지, 원나잇, 시한부, 왕족/귀족, 무심남, 짝사랑녀, 삼각관계, 애잔물",금,KAKAOPAGE


In [56]:
# BL 또는 GL 이 들어간 작품
blgl_mask = all_webtoons["genre"].fillna("").str.contains(r"\bBL\b|\bGL\b")
df_blgl = all_webtoons[blgl_mask]
df_blgl[df_blgl["is_adult"]==False]

,titleId,titleName,thumbnailUrl,is_adult,Writer,Painter,Original,synopsis,genre,day,provider
742,1768,타원을 그리는 법,https://kr-a.kakaopagecdn.com/P/C/1768/c1/2x/c...,False,섬멍,섬멍,NaN,오합지졸들과 함께\n회사의 비밀 프로젝트를 밝혀내고\n연인을 과거에서 구출하라!\n...,"기발한, 궁금하게 하는, 로맨스, GL, ROMANCE",월,KAKAO
1536,4463,연습생,https://kr-a.kakaopagecdn.com/P/C/4463/c1/2x/4...,False,마루,마루,NaN,굴곡진 인생에 재미를 잃은 소년 우주 앞에 나타난 \n가수 연습생 평화는 무엇보다도...,"설레는, 귀여운, 로맨스, BL, ROMANCE",목,KAKAO
1551,4480,목숨을 다해 꼬셔라,https://kr-a.kakaopagecdn.com/P/C/4480/c1/2x/d...,False,오청,오청,NaN,"외모와 능력, 분위기까지 갖춘 완벽한 남자 호연우. \n하지만 연애 상대로는 최악,...","설레는, 몰입되는, 로맨스, BL, ROMANCE",목,KAKAO
1748,4695,붉은 달밤의 끝자락에서,https://kr-a.kakaopagecdn.com/P/C/4695/c1/2x/9...,False,오공시,오공시,NaN,"저주받은 아이라 손가락질 받으며 외롭게 살아온 자운.\n\n어느 날, 괴질이 온 마...","에로틱한, 궁금하게 하는, 로맨스, BL, ROMANCE",토,KAKAO
1775,51478643,"독점계약입니다만,",//dn-img-page.kakao.com/download/resource?kid=...,False,검틱,검틱,NaN,고급 호텔 화장실에서 우연히 만나게 된 사회부 기자 황인찬과 유명 소설가 기태범. ...,"BL, 현대물, 오해물, 동거물, 트라우마, 능글, 냉정, 재력, 츤데레, 집착, ...",화,KAKAOPAGE
1803,53165109,별을 구속하겠습니다,//dn-img-page.kakao.com/download/resource?kid=...,False,텐센트동만,텐센트동만,텐센트동만,"까칠한 금수저 X 전직 경찰의 스타가 되기 위한 여정!\n경찰이었던 지은우, 철부지...","BL, 라이벌/앙숙, 두근거리는, 작화가 쩌는, 성장물, 강공, 연하공, 후회공, ...",수,KAKAOPAGE
1875,55310599,메리 수,//dn-img-page.kakao.com/download/resource?kid=...,False,나인썸머,리틀머드,NaN,"23세기, VR 기술이 눈부시게 성장하며 인류는 가상 기대에 돌입했다.\n\n가장 ...","BL, 가상현실, 현대물, 성장물, 막장, 전문직물, 우연한만남, 냉정, 무심, 집...",월,KAKAOPAGE
1960,57153091,비터 레몬,//dn-img-page.kakao.com/download/resource?kid=...,False,존생,팩둥,NaN,"“왜 이 녀석이 내 주변을 맴도는 거지? \n\n게다가, 학교 앞 럭셔리한 오피스텔...","BL, 현대물, 캠퍼스물, 동거물, 미인공, 츤데레수, 첫사랑, 친구>연인, 작화가 쩌는",토,KAKAOPAGE
1993,57777745,별과 별 사이,//dn-img-page.kakao.com/download/resource?kid=...,False,스리,스리,NaN,하나뿐인 형이 사고로 다쳤다.\n\n범인은 부잣집 아들에 준수한 외모로 학교의 유명...,"BL, 현대물, 학원물, 오해물, 착각물, 무심, 츤데레, 미남, 친구>연인",수,KAKAOPAGE
2031,58352233,영원한 계약,//dn-img-page.kakao.com/download/resource?kid=...,False,해진,해진,NaN,"[매주 수요일 업데이트, 5주 연재 후 1주 정기 휴재]\n\n\n“내가 널 따라온...","BL, 중세물, 판타지물, 환생물, 운명, 능글, 집착, 능력있는, 무심한, 삼각관...",수,KAKAOPAGE


In [17]:
mask_blgl = all_webtoons['genre'].str.contains('bl|gl', na=False, regex=True, case=False)
df_no_blgl = all_webtoons[~mask_blgl].copy()

In [18]:
# all_genres = (
#     all_webtoons["genre"][all_webtoons["is_adult"]==False]
#     .dropna()
#     .str.split(",")
#     .explode()
#     .str.strip()
#     .replace("", pd.NA)
#     .dropna()
# )

all_genres = (
    df_no_blgl["genre"][all_webtoons["is_adult"]==False]
    .dropna()
    .str.split(",")
    .explode()
    .str.strip()
    .replace("", pd.NA)
    .dropna()
)

unique_genres = all_genres.unique()

In [19]:
genre_df = pd.DataFrame({"raw_genre": sorted(unique_genres)})
genre_df.to_csv("all_raw_genres.csv", index=False)

In [3]:
genre_df = pd.read_csv("all_raw_genres.csv")

In [8]:
import pandas as pd
import numpy as np
import re

# 1. 데이터 로드 및 전처리 (이전과 동일)
file_path = 'all_raw_genres.csv' 
try:
    genre_df = pd.read_csv(file_path)
except: 
    print("파일을 찾을 수 없습니다.")
    exit()

if 'raw_genre' not in genre_df.columns:
    target_column = genre_df.columns[0]
else:
    target_column = 'raw_genre'

# BL/GL 및 중복 제거
stop_words = ['BL', 'GL', '비엘', '백합']
mask = genre_df[target_column].astype(str).apply(lambda x: any(stop in x.upper() for stop in stop_words))
filtered_df = genre_df[~mask].copy()
filtered_df = filtered_df.drop_duplicates(subset=[target_column]).dropna(subset=[target_column])
genres = filtered_df[target_column].tolist()
print(f"분석 대상 단어 개수: {len(genres)}")


# 2. 40개 목표 그룹과 매핑 키워드 정의 (도메인 지식 기반)
# 키워드 순서가 중요합니다. 구체적인 키워드(예: '오피스')를 먼저 배치해야 포괄적인 키워드(예: '로맨스')가 덮어쓰지 않습니다.

# [키워드]: 그룹 이름
MAPPING_RULES = {
    # 1. 핵심 메인 장르
    '액션|ACTION|무협|WUXIA': '1_액션_무협',
    '판타지|FANTASY|퓨전|SF|이세계|MAGIC|마법': '2_판타지_SF',
    '로맨스|ROMANCE|연애|사랑|PURE|퓨어|짝사랑': '3_로맨스_연애',
    '드라마|DRAMA|일상|DAILY|에피소드|힐링': '4_드라마_일상_힐링',
    '호러|HORROR|공포|오컬트|THRILL': '5_호러_공포_스릴러',
    '개그|COMIC|코믹|병맛|유쾌': '6_개그_코미디',
    
    # 2. 서브 장르 및 설정
    '빙의|환생|회귀|타임슬립|TS': '7_빙의_회귀_환생',
    '학교|학원|캠퍼스|청춘|SCHOOL': '8_학원_캠퍼스_청춘',
    '오피스|재벌|회사원|경영|사업': '9_오피스_재벌_경영',
    '전문직|가수|아이돌|연예계|의사|감독|PD|운동선수': '10_전문직_연예_스포츠',
    '역사|시대극|HISTORICAL|궁중|왕족|귀족|황제': '11_역사_시대극_궁중',
    '가족|육아|부모|자매|형제|우정': '12_가족_육아',
    '게임|시스템|던전|레벨업|VR|가상현실': '13_게임_시스템',
    '미스터리|추리|범죄|수사|법정|탐정': '14_미스터리_추리_범죄',
    '복수|후회|절망|처절|트라우마': '15_복수_후회_응징',
    
    # 3. 세부 설정 및 태그 (40개 그룹을 만들기 위해 세분화)
    '성장|성공|천재': '16_성장_성공',
    '다크|블랙|아포칼립스|재난|좀비': '17_다크_재난',
    'SF|미래|기갑|AI': '18_SF_미래',
    '액션판타지|SF액션|SCHOOL_ACTION_FANTASY': '19_액션_하이브리드',
    '동양풍|WUXIA|무협': '20_동양풍_무협', # 1번과 중복되는 WUXIA를 여기에도 배치하여 그룹화 보조
    '서양풍|판타지': '21_서양풍_판타지', # 2번과 중복되는 판타지를 여기에도 배치
    '아방|소심|순진|소녀|명랑|귀여운': '22_순수_발랄_청순',
    '계략|까칠|냉정|오만|나쁜|츤데레|소유욕': '23_계략_까칠_나쁜남녀',
    '군인|전쟁': '24_군인_전쟁',
    '음식|요리|먹방': '25_음식_요리',
    '능력|초능력|영웅|신화': '26_초능력_영웅',
    '아이돌|연예인|가수': '27_아이돌_가수',
    '연상|연하|나이차': '28_연상연하',
    '결혼|선결혼|정략결혼': '29_결혼_선결혼',
    '가이드버스|헌터|레이드': '30_헌터물_레이드',
    '여주중심|여주판타지': '31_여주_중심',
    '남주중심|소년물': '32_남주_중심',
    '4컷만화|웹툰': '33_단편_4컷',
    '작가|소설원작': '34_창작_원작',
    '공모전|최강자전|직행열차': '35_공모전_수상작',
    '정치|정치가': '36_정치_외교',
    '의사|의학': '37_의료_메디컬',
    '검|격투|무술': '38_격투_무술',
    '외국인|해외|혼혈': '39_해외_외국',
    '기타': '40_기타_분류불가',
}

# 3. 그룹 할당 로직
def assign_group(genre):
    # 단어 자체의 불필요한 공백 제거
    genre_clean = genre.strip()
    
    # 매핑 규칙을 순회하며 매칭
    for keywords, group_name in MAPPING_RULES.items():
        # 정규식 패턴을 사용하여 대소문자 구분 없이 매칭
        if re.search(keywords, genre_clean, re.IGNORECASE):
            return group_name
            
    # 어떤 그룹에도 매칭되지 않은 경우
    return '40_기타_분류불가'

filtered_df['cluster_name'] = filtered_df[target_column].apply(assign_group)

# 4. 결과 출력 및 요약
print("\n" + "="*60)
print("   ⭐ 최종 키워드 기반 장르 분류 결과 (40개 그룹)")
print("="*60)

# 그룹별 개수 요약
summary_df = filtered_df.groupby('cluster_name').size().reset_index(name='count')
summary_df = summary_df.sort_values(by='cluster_name').reset_index(drop=True)

# 그룹별 목록 출력
for index, row in summary_df.iterrows():
    group_name = row['cluster_name']
    group_count = row['count']
    
    # 해당 그룹에 속한 단어 목록 추출
    items = filtered_df[filtered_df['cluster_name'] == group_name][target_column].tolist()
    
    print(f"[{group_name}] (크기: {group_count})")
    content = ', '.join(items)
    if len(content) > 150:
        print(f"목록: {content[:150]} ... (총 {group_count}개)")
    else:
        print(f"목록: {content}")
    print("-" * 30)

# 5. 결과를 CSV 파일로 저장
output_file = 'final_40_grouped_genres.csv'
filtered_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n✅ 최종 결과가 '{output_file}' 파일로 저장되었습니다. 이제 40개의 그룹이 보장됩니다.")

분석 대상 단어 개수: 615

   ⭐ 최종 키워드 기반 장르 분류 결과 (40개 그룹)
[10_전문직_연예_스포츠] (크기: 10)
목록: PD/감독, 가수, 감독/스카우터, 아이돌, 연예계, 연예계물, 운동선수, 의사, 전문직, 전문직물
------------------------------
[11_역사_시대극_궁중] (크기: 4)
목록: HISTORICAL, 대체역사, 역사물, 왕족/귀족
------------------------------
[12_가족_육아] (크기: 7)
목록: 가족, 가족물, 가족후회물, 우정, 육아, 육아물, 형제/자매/남매
------------------------------
[13_게임_시스템] (크기: 7)
목록: 가상현실, 게임, 던전물, 데스게임, 레벨업물, 머니게임, 시스템
------------------------------
[14_미스터리_추리_범죄] (크기: 6)
목록: 미스터리, 범죄, 범죄스릴러물, 수사물, 추리물, 탐정
------------------------------
[15_복수_후회_응징] (크기: 9)
목록: 복수, 복수극, 복수물, 절망적인, 처절한, 트라우마, 후회남, 후회녀, 후회물
------------------------------
[16_성장_성공] (크기: 7)
목록: 성공성장물, 성장, 성장로판, 성장물, 성장하는, 스포츠성장, 천재
------------------------------
[17_다크_재난] (크기: 5)
목록: 다크히어로, 블랙코미디, 아포칼립스, 재난물, 좀비
------------------------------
[18_SF_미래] (크기: 4)
목록: AI, 기갑, 미래, 미래시대
------------------------------
[1_액션_무협] (크기: 15)
목록: ACTION, ACTION_WUXIA, SCHOOL_ACTION_FANTASY, SF액션, 객잔무협, 무협, 무협/사극, 무협물, 액션, 액션/무협, 액션아포칼립스, 액션판타지, 정

In [ ]:
filtered_df

In [9]:
import pandas as pd
import numpy as np
import re

# 1. 데이터 로드 및 전처리 (이전과 동일)
file_path = 'final_40_grouped_genres.csv'  # 이전에 저장된 파일을 사용하거나 원본 파일명('all_raw_genres.csv')으로 변경 가능
try:
    # 사용자님이 마지막으로 제공하신 파일명을 사용합니다.
    df = pd.read_csv(file_path) 
except: 
    print("파일을 찾을 수 없습니다. 원본 파일('all_raw_genres.csv' 등)명을 확인해 주세요.")
    exit()

if 'raw_genre' not in df.columns:
    # 데이터프레임의 첫 번째 컬럼을 사용합니다.
    target_column = df.columns[0]
else:
    target_column = 'raw_genre'

# BL/GL 및 중복 제거 (원본 데이터에 대해 다시 실행)
stop_words = ['BL', 'GL', '비엘', '백합']
mask = df[target_column].astype(str).apply(lambda x: any(stop in x.upper() for stop in stop_words))
filtered_df = df[~mask].copy()
filtered_df = filtered_df.drop_duplicates(subset=[target_column]).dropna(subset=[target_column])
genres = filtered_df[target_column].tolist()
print(f"분석 대상 단어 개수: {len(genres)}")


# 2. 40개 목표 그룹과 매핑 키워드 정의 (V7 최종 완성 규칙)

MAPPING_RULES = {
    # 1. 핵심 메인 장르
    '액션|ACTION|무협|WUXIA': '1_액션_무협',
    '판타지|FANTASY|퓨전|SF|이세계|MAGIC|마법': '2_판타지_SF',
    # V7: 썸, 치정, 친구>연인, 오래된연인, 혐관 추가
    '로맨스|ROMANCE|연애|사랑|PURE|퓨어|짝사랑|치정|친구>연인|썸|혐관|오래된연인': '3_로맨스_연애', 
    # V7: 치유, 감성, 평온한, 코믹/일상, 직업드라마 추가
    '드라마|DRAMA|일상|DAILY|에피소드|힐링|치유|감성|평온한|코믹/일상|직업드라마': '4_드라마_일상_힐링', 
    # V7: 크리처, 크툴루, 치명적, 오컬트, 괴담 추가
    '호러|HORROR|공포|오컬트|THRILL|크리처|크툴루|치명적|괴담': '5_호러_공포_스릴러', 
    # V7: 4차원, 컷툰, 유쾌한, 병맛 추가
    '개그|COMIC|코믹|병맛|유쾌|4차원|컷툰': '6_개그_코미디', 
    
    # 2. 서브 장르 및 설정
    # V7: 영혼체인지 추가
    '빙의|환생|회귀|타임슬립|TS|영혼체인지': '7_빙의_회귀_환생', 
    '학교|학원|캠퍼스|청춘|SCHOOL': '8_학원_캠퍼스_청춘',
    # V7: 취업, 회사 추가
    '오피스|재벌|회사원|경영|사업|취업|회사': '9_오피스_재벌_경영', 
    # V7: 축구, SPORTS 등 스포츠 단어 대거 추가
    '전문직|가수|아이돌|연예계|의사|감독|PD|운동선수|축구|야구|농구|배구|SPORTS|인플루언서|BJ': '10_전문직_연예_스포츠', 
    '역사|시대극|HISTORICAL|궁중|왕족|귀족|황제': '11_역사_시대극_궁중',
    '가족|육아|부모|자매|형제|우정': '12_가족_육아',
    # V7: 탑등반, 플레이어 추가
    '게임|시스템|던전|레벨업|VR|가상현실|탑등반|플레이어': '13_게임_시스템', 
    # V7: 미스테리, 서스펜스, 경찰 추가
    '미스터리|미스테리|추리|범죄|수사|법정|탐정|형사|경찰|서스펜스': '14_미스터리_추리_범죄', 
    '복수|후회|절망|처절|트라우마': '15_복수_후회_응징',
    
    # 3. 세부 설정 및 태그
    '성장|성공|천재': '16_성장_성공',
    '다크|블랙|아포칼립스|재난|좀비': '17_다크_재난',
    'SF|미래|기갑|AI': '18_SF_미래',
    '액션판타지|SF액션|SCHOOL_ACTION_FANTASY': '19_액션_하이브리드',
    '동양풍|WUXIA|무협': '20_동양풍_무협', 
    '서양풍|판타지': '21_서양풍_판타지', 
    # V7: 털털녀 추가
    '아방|소심|순진|소녀|명랑|귀여운|털털녀': '22_순수_발랄_청순', 
    # V7: 츤데레, 카리스마, 짐승남, 집착남 추가
    '계략|까칠|냉정|오만|나쁜|츤데레|소유욕|카리스마|짐승남|집착남|빌런': '23_계략_까칠_나쁜남녀', 
    '군인|전쟁': '24_군인_전쟁',
    '음식|요리|먹방': '25_음식_요리',
    # V7: 힐러, 용사, 마왕 추가
    '능력|초능력|영웅|신화|힐러|용사|마왕': '26_초능력_영웅', 
    '아이돌|연예인|가수': '27_아이돌_가수',
    '연상|연하|나이차': '28_연상연하',
    '결혼|선결혼|정략결혼': '29_결혼_선결혼',
    '가이드버스|헌터|레이드': '30_헌터물_레이드',
    '여주중심|여주판타지': '31_여주_중심',
    '남주중심|소년물': '32_남주_중심',
    # V7: 옴니버스 추가
    '4컷만화|웹툰|옴니버스': '33_단편_4컷', 
    '작가|소설원작': '34_창작_원작',
    '공모전|최강자전|직행열차': '35_공모전_수상작',
    '정치|정치가': '36_정치_외교',
    '의사|의학': '37_의료_메디컬',
    '검|격투|무술': '38_격투_무술',
    '외국인|해외|혼혈': '39_해외_외국',
    '기타': '40_기타_분류불가', # 이 그룹은 이제 거의 비게 될 것입니다.
}

# 3. 그룹 할당 로직 (이전과 동일)
def assign_group(genre):
    genre_clean = genre.strip()
    
    for keywords, group_name in MAPPING_RULES.items():
        if re.search(keywords, genre_clean, re.IGNORECASE):
            return group_name
            
    return '40_기타_분류불가'

filtered_df['cluster_name'] = filtered_df[target_column].apply(assign_group)

# 4. 결과 출력 및 요약 (이전과 동일)
print("\n" + "="*60)
print("   ⭐ 최종 키워드 기반 장르 분류 결과 (40개 그룹, V7 최종)")
print("="*60)

summary_df = filtered_df.groupby('cluster_name').size().reset_index(name='count')
summary_df = summary_df.sort_values(by='cluster_name').reset_index(drop=True)

final_unclassified_count = 0
for index, row in summary_df.iterrows():
    group_name = row['cluster_name']
    group_count = row['count']
    
    items = filtered_df[filtered_df['cluster_name'] == group_name][target_column].tolist()
    
    print(f"[{group_name}] (크기: {group_count})")
    content = ', '.join(items)
    if len(content) > 150:
        print(f"목록: {content[:150]} ... (총 {group_count}개)")
    else:
        print(f"목록: {content}")
    print("-" * 30)
    
    if group_name == '40_기타_분류불가':
        final_unclassified_count = group_count

print(f"\n✅ '40_기타_분류불가' 그룹에 최종적으로 남은 단어 수: {final_unclassified_count}개")

# 5. 결과를 CSV 파일로 저장
output_file = 'final_40_grouped_genres_V7_FINAL.csv'
filtered_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n✅ 최종 결과가 '{output_file}' 파일로 저장되었습니다.")

분석 대상 단어 개수: 615

   ⭐ 최종 키워드 기반 장르 분류 결과 (40개 그룹, V7 최종)
[10_전문직_연예_스포츠] (크기: 16)
목록: BJ, PD/감독, 가수, 감독/스카우터, 농구, 아이돌, 야구, 야구선수, 연예계, 연예계물, 운동선수, 의사, 인플루언서, 전문직, 전문직물, 축구
------------------------------
[11_역사_시대극_궁중] (크기: 4)
목록: HISTORICAL, 대체역사, 역사물, 왕족/귀족
------------------------------
[12_가족_육아] (크기: 7)
목록: 가족, 가족물, 가족후회물, 우정, 육아, 육아물, 형제/자매/남매
------------------------------
[13_게임_시스템] (크기: 9)
목록: 가상현실, 게임, 던전물, 데스게임, 레벨업물, 머니게임, 시스템, 탑등반, 플레이어
------------------------------
[14_미스터리_추리_범죄] (크기: 11)
목록: 경찰, 미스터리, 미스테리, 미스테리한, 범죄, 범죄스릴러물, 서스펜스, 수사물, 추리물, 탐정, 형사
------------------------------
[15_복수_후회_응징] (크기: 9)
목록: 복수, 복수극, 복수물, 절망적인, 처절한, 트라우마, 후회남, 후회녀, 후회물
------------------------------
[16_성장_성공] (크기: 7)
목록: 성공성장물, 성장, 성장로판, 성장물, 성장하는, 스포츠성장, 천재
------------------------------
[17_다크_재난] (크기: 5)
목록: 다크히어로, 블랙코미디, 아포칼립스, 재난물, 좀비
------------------------------
[18_SF_미래] (크기: 4)
목록: AI, 기갑, 미래, 미래시대
------------------------------
[1_액션_무협] (크기: 15)
목록: ACTION, ACTION_WUXIA, SCHO

In [3]:
import pandas as pd
import numpy as np
import re

# 1. 데이터 로드 및 전처리 (이전과 동일)
file_path = 'final_40_grouped_genres_V7_FINAL.csv'
try:
    df = pd.read_csv(file_path)
except: 
    print("파일을 찾을 수 없습니다. 원본 파일명을 확인해 주세요.")
    exit()

if 'raw_genre' not in df.columns:
    target_column = df.columns[0]
else:
    target_column = 'raw_genre'

# BL/GL 및 중복 제거 (원본 데이터에 대해 다시 실행)
stop_words = ['BL', 'GL', '비엘', '백합']
mask = df[target_column].astype(str).apply(lambda x: any(stop in x.upper() for stop in stop_words))
filtered_df = df[~mask].copy()
filtered_df = filtered_df.drop_duplicates(subset=[target_column]).dropna(subset=[target_column])
genres = filtered_df[target_column].tolist()
print(f"분석 대상 단어 개수: {len(genres)}")


# 2. 40개 목표 그룹과 매핑 키워드 정의 (V8 극단적 흡수 규칙)
# V8: V7의 규칙을 유지하고, 40번 그룹에 남아있던 단어들을 최대한 흡수하도록 규칙을 추가함.

MAPPING_RULES_V8 = {
    # 1. 핵심 메인 장르
    '액션|ACTION|무협|WUXIA|검|격투|무술|객잔무협|무림|낭인|살수|협객|정통무협|화산': '1_액션_무협', # 무협/격투 관련 단어 추가
    '판타지|FANTASY|퓨전|SF|이세계|MAGIC|마법|오컬트판타지|게임판타지|이세계요리|역사판타지': '2_판타지_SF', 
    '로맨스|ROMANCE|연애|사랑|PURE|퓨어|짝사랑|치정|친구>연인|썸|혐관|오래된연인|비밀연애|배틀연애|전남친|삼각관계|할리퀸': '3_로맨스_연애', # 관계 단어 추가
    '드라마|DRAMA|일상|DAILY|에피소드|힐링|치유|감성|평온한|코믹/일상|직업드라마|잔잔|신파|고자극드라마': '4_드라마_일상_힐링', # 감정/상태 단어 추가
    '호러|HORROR|공포|오컬트|THRILL|크리처|크툴루|치명적|괴담|귀신|등골오싹|감염': '5_호러_공포_스릴러', 
    '개그|COMIC|코믹|병맛|유쾌|4차원|컷툰|노블코믹|블랙코미디|열혈병맛개그': '6_개그_코미디', 
    
    # 2. 서브 장르 및 설정
    '빙의|환생|회귀|타임슬립|TS|영혼체인지|전생': '7_빙의_회귀_환생', 
    '학교|학원|캠퍼스|청춘|SCHOOL|하이틴|학생': '8_학원_캠퍼스_청춘',
    '오피스|재벌|회사원|경영|사업|취업|회사|상인|자본주의|머니게임': '9_오피스_재벌_경영', 
    '전문직|가수|아이돌|연예계|의사|감독|PD|운동선수|축구|야구|농구|배구|SPORTS|인플루언서|BJ|배우|방송|매니저|디자이너|소방관|법조인|검사': '10_전문직_연예_스포츠', # 전문직 대폭 추가
    '역사|시대극|HISTORICAL|궁중|왕족|귀족|황제|대체역사|퓨전사극': '11_역사_시대극_궁중',
    '가족|육아|부모|자매|형제|우정|베이비메신저|형제자매남매': '12_가족_육아',
    '게임|시스템|던전|레벨업|VR|가상현실|탑등반|플레이어|고인물|데스게임|배틀물': '13_게임_시스템', # 게임/경쟁 단어 추가
    '미스터리|미스테리|추리|범죄|수사|법정|탐정|형사|경찰|서스펜스|느와르|피카레스크|조직|암흑가': '14_미스터리_추리_범죄', # 느와르/범죄 단어 추가
    '복수|후회|절망|처절|트라우마|복수극|분노유발|암투': '15_복수_후회_응징',
    
    # 3. 세부 설정 및 태그
    '성장|성공|천재|노력|성장성공물|인생역전': '16_성장_성공',
    '다크|블랙|아포칼립스|재난|좀비|밀리터리|전쟁': '17_다크_재난', # 군사/재난 단어 흡수
    'SF|미래|기갑|AI|메카닉물': '18_SF_미래', # 메카닉물 추가
    '액션판타지|SF액션|SCHOOL_ACTION_FANTASY': '19_액션_하이브리드',
    '동양풍|WUXIA|무협': '20_동양풍_무협', 
    '서양풍|판타지|중세': '21_서양풍_판타지', # 중세 추가
    # V8: 순수함/발랄함 관련 단어 대거 추가
    '아방|소심|순진|소녀|명랑|귀여운|털털녀|발랄|사랑스러운|명랑녀|햇살|애교': '22_순수_발랄_청순', 
    # V8: 까칠함/악역/강렬한 캐릭터 관련 단어 대거 추가
    '계략|까칠|냉정|오만|나쁜|츤데레|소유욕|카리스마|짐승남|집착남|빌런|악녀|악역|절륜|오빠|여공남수|차도남|능글|도도|폭스': '23_계략_까칠_나쁜남녀', 
    '군인|전쟁': '24_군인_전쟁', # 17번과 겹치지만 혹시 모를 분리를 위해 유지
    '음식|요리|먹방|식욕|요리사': '25_음식_요리',
    # V8: 능력자 역할 단어 추가
    '능력|초능력|영웅|신화|힐러|용사|마왕|네크로맨서|마검사|영지|이능력|이능력배틀물': '26_초능력_영웅', 
    '아이돌|연예인|가수': '27_아이돌_가수',
    '연상|연하|나이차': '28_연상연하',
    '결혼|선결혼|정략결혼': '29_결혼_선결혼',
    '가이드버스|헌터|레이드': '30_헌터물_레이드',
    '여주중심|여주판타지|걸크러쉬|군림녀': '31_여주_중심', # 걸크러쉬 추가
    '남주중심|소년물|소년왕도물': '32_남주_중심',
    '4컷만화|웹툰|옴니버스': '33_단편_4컷', 
    '작가|소설원작|창작': '34_창작_원작',
    '공모전|최강자전|직행열차': '35_공모전_수상작',
    '정치|정치가|외교': '36_정치_외교',
    '의사|의학|메디컬': '37_의료_메디컬',
    '검|격투|무술': '38_격투_무술',
    '외국인|해외|혼혈': '39_해외_외국',
    '가벼운|독자PICK|요즘핫한추천작|분량많은|재미있는|긴장감있는|몰입되는|무서운|기타': '40_기타_분류불가', # 속성/추천 단어만 남김
}

# 3. 그룹 할당 로직 (이전과 동일)
def assign_group(genre):
    genre_clean = genre.strip()
    
    # MAPPING_RULES_V8을 사용
    for keywords, group_name in MAPPING_RULES_V8.items(): 
        if re.search(keywords, genre_clean, re.IGNORECASE):
            return group_name
            
    return '40_기타_분류불가'

filtered_df['cluster_name'] = filtered_df[target_column].apply(assign_group)

# 4. 결과 출력 및 요약 (이전과 동일)
print("\n" + "="*60)
print("   ⭐ 최종 키워드 기반 장르 분류 결과 (40개 그룹, V8 최종 흡수)")
print("="*60)

summary_df = filtered_df.groupby('cluster_name').size().reset_index(name='count')
summary_df = summary_df.sort_values(by='cluster_name').reset_index(drop=True)

final_unclassified_count = 0
for index, row in summary_df.iterrows():
    group_name = row['cluster_name']
    group_count = row['count']
    
    items = filtered_df[filtered_df['cluster_name'] == group_name][target_column].tolist()
    
    print(f"[{group_name}] (크기: {group_count})")
    content = ', '.join(items)
    if len(content) > 150:
        print(f"목록: {content[:150]} ... (총 {group_count}개)")
    else:
        print(f"목록: {content}")
    print("-" * 30)
    
    if group_name == '40_기타_분류불가':
        final_unclassified_count = group_count

print(f"\n✅ '40_기타_분류불가' 그룹에 최종적으로 남은 단어 수: {final_unclassified_count}개")
print(f"✅ 이 코드를 실행하면 미분류 단어가 최소화됩니다.")

# # 5. 결과를 CSV 파일로 저장
output_file = 'final_40_grouped_genres_V8_FINAL.csv'
filtered_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n✅ 최종 결과가 '{output_file}' 파일로 저장되었습니다.")

분석 대상 단어 개수: 615

   ⭐ 최종 키워드 기반 장르 분류 결과 (40개 그룹, V8 최종 흡수)
[10_전문직_연예_스포츠] (크기: 22)
목록: BJ, PD/감독, 가수, 감독/스카우터, 농구, 디자이너, 매니저, 방송, 배우, 법조인, 소방관, 아이돌, 야구, 야구선수, 연예계, 연예계물, 운동선수, 의사, 인플루언서, 전문직, 전문직물, 축구
------------------------------
[11_역사_시대극_궁중] (크기: 4)
목록: HISTORICAL, 대체역사, 역사물, 왕족/귀족
------------------------------
[12_가족_육아] (크기: 8)
목록: 가족, 가족물, 가족후회물, 베이비메신저, 우정, 육아, 육아물, 형제/자매/남매
------------------------------
[13_게임_시스템] (크기: 11)
목록: 가상현실, 게임, 고인물, 던전물, 데스게임, 레벨업물, 배틀물, 시스템, 이능력배틀물, 탑등반, 플레이어
------------------------------
[14_미스터리_추리_범죄] (크기: 14)
목록: 경찰, 느와르, 미스터리, 미스테리, 미스테리한, 범죄, 범죄스릴러물, 서스펜스, 수사물, 조직/암흑가, 추리물, 탐정, 피카레스크, 형사
------------------------------
[15_복수_후회_응징] (크기: 11)
목록: 복수, 복수극, 복수물, 분노유발, 암투물, 절망적인, 처절한, 트라우마, 후회남, 후회녀, 후회물
------------------------------
[16_성장_성공] (크기: 9)
목록: 노력, 성공성장물, 성장, 성장로판, 성장물, 성장하는, 스포츠성장, 인생역전, 천재
------------------------------
[17_다크_재난] (크기: 7)
목록: 다크히어로, 밀리터리, 아포칼립스, 재난물, 전쟁, 전쟁물, 좀비
------------------------------
[18_SF_미래] (크기: 5

In [4]:
import pandas as pd
import numpy as np
import re

# 1. 데이터 로드 (사용자님이 제공한 V8 파일 기반으로 V9 재실행)
file_path = 'final_40_grouped_genres_V8_FINAL.csv'
try:
    df = pd.read_csv(file_path)
    target_column = 'raw_genre' if 'raw_genre' in df.columns else df.columns[0]
except Exception as e: 
    print(f"파일 로드 실패: {e}")
    exit()

# BL/GL 및 중복 제거
stop_words = ['BL', 'GL', '비엘', '백합']
mask = df[target_column].astype(str).apply(lambda x: any(stop in x.upper() for stop in stop_words))
filtered_df = df[~mask].copy()
filtered_df = filtered_df.drop_duplicates(subset=[target_column]).dropna(subset=[target_column])
genres = filtered_df[target_column].tolist()
print(f"분석 대상 단어 개수: {len(genres)}")

# 2. 40개 목표 그룹과 매핑 키워드 정의 (V9 최종 규칙 사용)
MAPPING_RULES_V9 = {
    '액션|ACTION|무협|WUXIA|검|격투|무술|객잔무협|무림|낭인|살수|협객|정통무협|화산': '1_액션_무협',
    '판타지|FANTASY|퓨전|SF|이세계|MAGIC|마법|오컬트판타지|게임판타지|현대판타지|환타지|초월적존재': '2_판타지_SF', # 초월적존재 추가
    '로맨스|ROMANCE|연애|사랑|PURE|퓨어|짝사랑|치정|친구>연인|썸|혐관|오래된연인|비밀연애|배틀연애|전남친|삼각관계|사제지간|주종관계|갑을관계|오해|우연한만남|쌍방삽질|경쟁구도|철벽남': '3_로맨스_연애', 
    '드라마|DRAMA|일상|DAILY|에피소드|힐링|치유|감성|평온한|코믹/일상|직업드라마|잔잔|신파|고자극드라마|가벼운|공감|따뜻한|흐뭇한|웃기는|재미있는|신선한|센스있는|청춘드라마|시월드': '4_드라마_일상_힐링', 
    '호러|HORROR|공포|오컬트|THRILL|크리처|크툴루|치명적|괴담|귀신|등골오싹|감염|저승|종교|퇴마|압도되는|긴장감있는': '5_호러_공포_스릴러', 
    '개그|COMIC|코믹|병맛|유쾌|4차원|컷툰|노블코믹|블랙코미디|열혈병맛개그': '6_개그_코미디', 
    '빙의|환생|회귀|타임슬립|TS|영혼체인지|전생': '7_빙의_회귀_환생', 
    '학교|학원|캠퍼스|청춘|SCHOOL|하이틴|학생|학교생활|인소감성': '8_학원_캠퍼스_청춘',
    '오피스|재벌|회사원|경영|사업|취업|회사|상인|자본주의|머니게임': '9_오피스_재벌_경영', 
    '전문직|가수|아이돌|연예계|의사|감독|PD|운동선수|축구|야구|농구|배구|SPORTS|인플루언서|BJ|배우|방송|매니저|디자이너|소방관|법조인|검사': '10_전문직_연예_스포츠', 
    '역사|시대극|HISTORICAL|궁중|왕족|귀족|황제|대체역사|퓨전사극|궁|폐하|후궁|가상시대물': '11_역사_시대극_궁중', # 가상시대물 추가
    '가족|육아|부모|자매|형제|우정|베이비메신저|형제자매남매': '12_가족_육아',
    '게임|시스템|던전|레벨업|VR|가상현실|탑등반|플레이어|고인물|데스게임|배틀물': '13_게임_시스템', 
    '미스터리|미스테리|추리|범죄|수사|법정|탐정|형사|경찰|서스펜스|느와르|피카레스크|조직|암흑가|반전물': '14_미스터리_추리_범죄', 
    '복수|후회|절망|처절|트라우마|복수극|분노유발|암투|가슴 먹먹한|슬픈|아련한|답답한|짠내나는|배신|복수대행|통쾌한': '15_복수_후회_응징',
    '성장|성공|천재|노력|성장성공물|인생역전': '16_성장_성공',
    '다크|블랙|아포칼립스|재난|좀비|밀리터리|전쟁|군대물': '17_다크_재난',
    'SF|미래|기갑|AI|메카닉물|인공지능': '18_SF_미래', # 인공지능 추가
    '액션판타지|SF액션|SCHOOL_ACTION_FANTASY': '19_액션_하이브리드',
    '동양풍|WUXIA|무협': '20_동양풍_무협', 
    '서양풍|판타지|중세': '21_서양풍_판타지', 
    '아방|소심|순진|소녀|명랑|귀여운|털털녀|발랄|사랑스러운|명랑녀|햇살|애교|외유내강': '22_순수_발랄_청순', 
    '계략|까칠|냉정|오만|나쁜|츤데레|소유욕|카리스마|짐승남|집착남|빌런|악녀|악역|절륜|오빠|여공남수|차도남|능글|도도|폭스|대형견남|폭군|남장여자|후계자': '23_계략_까칠_나쁜남녀', 
    '군인': '24_군인_전쟁', 
    '음식|요리|먹방|식욕|요리사': '25_음식_요리',
    '능력|초능력|영웅|신화|힐러|용사|마왕|네크로맨서|마검사|영지|이능력|이능력배틀물|악마|천사|신|히어로물': '26_초능력_영웅', # 히어로물 추가
    '아이돌|연예인|가수': '27_아이돌_가수',
    '연상|연하|나이차': '28_연상연하',
    '결혼|선결혼|정략결혼': '29_결혼_선결혼',
    '가이드버스|헌터|레이드': '30_헌터물_레이드',
    '여주중심|여주판타지|걸크러쉬|군림녀|우월녀': '31_여주_중심', 
    '남주중심|소년물|소년왕도물': '32_남주_중심',
    '4컷만화|웹툰|옴니버스': '33_단편_4컷', 
    '작가|소설원작|창작': '34_창작_원작',
    '공모전|최강자전|직행열차': '35_공모전_수상작',
    '정치|정치가|외교': '36_정치_외교',
    '의사|의학|메디컬': '37_의료_메디컬',
    '검|격투|무술': '38_격투_무술',
    '외국인|해외|혼혈': '39_해외_외국',
    # 최종적으로 남은 평가/속성 단어만 40번으로 설정
    '독자PICK|요즘핫한추천작|초강력추천|명작|작화가_좋은|재미있는|궁금하게 하는|가슴 뛰는|센스있는|선이 굵은|섬세한|흥미진진한|몰입되는|반전있는|분량많은|테이밍|후유증|SENSIBILITY|감정적인': '40_기타_분류불가',
}

# 3. 그룹 할당 로직
def assign_group(genre):
    genre_clean = genre.strip()
    
    for keywords, group_name in MAPPING_RULES_V9.items(): 
        if re.search(keywords, genre_clean, re.IGNORECASE):
            return group_name
            
    return '40_기타_분류불가'

filtered_df['cluster_name'] = filtered_df[target_column].apply(assign_group)

# 4. 결과 출력 및 요약
print("\n" + "="*60)
print("   ⭐ 최종 키워드 기반 장르 분류 결과 (40개 그룹, V9 최종 보완)")
print("="*60)

summary_df = filtered_df.groupby('cluster_name').size().reset_index(name='count')
summary_df = summary_df.sort_values(by='cluster_name').reset_index(drop=True)

final_unclassified_count = 0
for index, row in summary_df.iterrows():
    group_name = row['cluster_name']
    group_count = row['count']
    
    items = filtered_df[filtered_df['cluster_name'] == group_name][target_column].tolist()
    
    if group_name == '40_기타_분류불가':
        print(f"[{group_name}] (크기: {group_count})")
        print(f"목록: {', '.join(items)}")
        print("-" * 30)
        final_unclassified_count = group_count
        continue
    
    print(f"[{group_name}] (크기: {group_count})")
    content = ', '.join(items)
    if len(content) > 150:
        print(f"목록: {content[:150]} ... (총 {group_count}개)")
    else:
        print(f"목록: {content}")
    print("-" * 30)

print(f"\n✅ '40_기타_분류불가' 그룹에 최종적으로 남은 단어 수: {final_unclassified_count}개")

# 5. 결과를 CSV 파일로 저장
output_file = 'final_40_grouped_genres_V9_FINAL.csv'
filtered_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n✅ 최종 결과가 '{output_file}' 파일로 저장되었습니다.")

분석 대상 단어 개수: 615

   ⭐ 최종 키워드 기반 장르 분류 결과 (40개 그룹, V9 최종 보완)
[10_전문직_연예_스포츠] (크기: 22)
목록: BJ, PD/감독, 가수, 감독/스카우터, 농구, 디자이너, 매니저, 방송, 배우, 법조인, 소방관, 아이돌, 야구, 야구선수, 연예계, 연예계물, 운동선수, 의사, 인플루언서, 전문직, 전문직물, 축구
------------------------------
[11_역사_시대극_궁중] (크기: 8)
목록: HISTORICAL, 가상시대물, 궁, 궁금하게 하는, 남궁, 대체역사, 역사물, 왕족/귀족
------------------------------
[12_가족_육아] (크기: 8)
목록: 가족, 가족물, 가족후회물, 베이비메신저, 우정, 육아, 육아물, 형제/자매/남매
------------------------------
[13_게임_시스템] (크기: 11)
목록: 가상현실, 게임, 고인물, 던전물, 데스게임, 레벨업물, 배틀물, 시스템, 이능력배틀물, 탑등반, 플레이어
------------------------------
[14_미스터리_추리_범죄] (크기: 15)
목록: 경찰, 느와르, 미스터리, 미스테리, 미스테리한, 반전물, 범죄, 범죄스릴러물, 서스펜스, 수사물, 조직/암흑가, 추리물, 탐정, 피카레스크, 형사
------------------------------
[15_복수_후회_응징] (크기: 16)
목록: 가슴 먹먹한, 답답한, 복수, 복수극, 복수물, 분노유발, 슬픈, 아련한, 암투물, 절망적인, 처절한, 통쾌한, 트라우마, 후회남, 후회녀, 후회물
------------------------------
[16_성장_성공] (크기: 9)
목록: 노력, 성공성장물, 성장, 성장로판, 성장물, 성장하는, 스포츠성장, 인생역전, 천재
------------------------------
[17_다크_재난] (크기: 8)
목록: 군대물, 다크히어로, 밀리터리, 아포칼립스, 재난물, 

In [6]:
import pandas as pd
import numpy as np
import re

# 1. 데이터 로드 (사용자님이 제공한 V9 파일 기반으로 V11 재실행)
file_path = 'final_40_grouped_genres_V9_FINAL.csv'
try:
    df = pd.read_csv(file_path)
    # raw_genre 컬럼을 사용합니다.
    target_column = 'raw_genre' 
except Exception as e: 
    print(f"파일 로드 실패: {e}")
    exit()

# BL/GL 및 중복 제거
stop_words = ['BL', 'GL', '비엘', '백합']
mask = df[target_column].astype(str).apply(lambda x: any(stop in x.upper() for stop in stop_words))
filtered_df = df[~mask].copy()
filtered_df = filtered_df.drop_duplicates(subset=[target_column]).dropna(subset=[target_column])
genres = filtered_df[target_column].tolist()
print(f"분석 대상 단어 개수: {len(genres)}")

# 2. 40개 목표 그룹과 매핑 키워드 정의 (V11 최종 규칙: 195개 잔여 단어 흡수)
MAPPING_RULES_V11 = {
    # 1. 핵심 메인 장르 & 무협 보강
    '액션|ACTION|무협|WUXIA|검|격투|무술|객잔무협|무림|낭인|살수|협객|정통무협|화산|곤륜|무공|천마|정파|사파|표국|창|학원액션|역동적인': '1_액션_무협', 
    # 2. 판타지/SF 보강
    '판타지|FANTASY|퓨전|SF|이세계|MAGIC|마법|오컬트판타지|게임판타지|현대판타지|환타지|초월적존재|테이밍|이종족|인외존재|정령|정령술사|세계관|차원이동|차원이동물|판무|학원/판타지': '2_판타지_SF', 
    # 3. 로맨스/연애 대폭 보강
    '로맨스|ROMANCE|연애|사랑|PURE|퓨어|짝사랑|치정|친구>연인|썸|혐관|오래된연인|비밀연애|배틀연애|전남친|삼각관계|사제지간|주종관계|갑을관계|오해|우연한만남|쌍방삽질|경쟁구도|철벽남|계약관계|금단의관계|맞선관계|원나잇|선배|소꿉친구|재회|재회물|친구|철벽녀|설레는|설렘폭발|러브코미디|러블리|로맨틱코미디|로맨틱코미디물|혐관로맨스|캠퍼스로맨스|오피스로맨스|현대로맨스|청춘로맨스|학원로맨스|학원로맨스물|에로틱한|끌리는': '3_로맨스_연애', 
    # 4. 드라마/일상/힐링 보강
    '드라마|DRAMA|일상|DAILY|에피소드|힐링|치유|감성|평온한|코믹/일상|직업드라마|잔잔|신파|고자극드라마|가벼운|공감|따뜻한|흐뭇한|웃기는|재미있는|청춘드라마|시월드|가슴 뛰는|SENSIBILITY|섬세한|감동적인|눈물샘자극|애잔물|시리어스|진중한|투병|하이퍼리얼리즘|일상물|과거|무해한': '4_드라마_일상_힐링', # '무해한', '과거' 추가
    # 5. 호러/공포/스릴러 보강
    '호러|HORROR|공포|오컬트|THRILL|크리처|크툴루|치명적|괴담|귀신|등골오싹|감염|저승|종교|퇴마|압도되는|긴장감있는|흥미진진한|고자극스릴러|스릴 넘치는|스릴러|무서운|연쇄살인': '5_호러_공포_스릴러', 
    # 6. 개그/코미디 보강
    '개그|COMIC|코믹|병맛|유쾌|4차원|컷툰|노블코믹|블랙코미디|열혈병맛개그|유머러스한': '6_개그_코미디', 
    
    # 7. 빙의/회귀/환생 보강
    '빙의|환생|회귀|타임슬립|TS|영혼체인지|전생|귀환|환골탈태': '7_빙의_회귀_환생', 
    # 8. 학원/캠퍼스/청춘 보강
    '학교|학원|캠퍼스|청춘|SCHOOL|하이틴|학생|학교생활|인소감성|아카데미물|입시|동아리': '8_학원_캠퍼스_청춘', 
    '오피스|재벌|회사원|경영|사업|취업|회사|상인|자본주의|머니게임|법조계': '9_오피스_재벌_경영', 
    '전문직|가수|아이돌|연예계|의사|감독|PD|운동선수|축구|야구|농구|배구|SPORTS|인플루언서|BJ|배우|방송|매니저|디자이너|소방관|법조인|검사|기타 스포츠|스포츠 선수|스포츠물': '10_전문직_연예_스포츠', 
    '역사|시대극|HISTORICAL|궁중|왕족|귀족|황제|대체역사|퓨전사극|궁|폐하|후궁|가상시대물|시대물': '11_역사_시대극_궁중', 
    '가족|육아|부모|자매|형제|우정|베이비메신저|형제자매남매': '12_가족_육아',
    '게임|시스템|던전|레벨업|VR|가상현실|탑등반|플레이어|고인물|데스게임|배틀물|상태창|지존': '13_게임_시스템', 
    '미스터리|미스테리|추리|범죄|수사|법정|탐정|형사|경찰|서스펜스|느와르|피카레스크|조직|암흑가|반전물|괴도|음모|사회고발': '14_미스터리_추리_범죄', 
    '복수|후회|절망|처절|트라우마|복수극|분노유발|암투|가슴 먹먹한|슬픈|아련한|답답한|짠내나는|배신|복수대행|통쾌한|후유증|참교육|피폐물|막장': '15_복수_후회_응징', 
    '성장|성공|천재|노력|성장성공물|인생역전': '16_성장_성공',
    '다크|블랙|아포칼립스|재난|좀비|밀리터리|전쟁|군대물|생존|서바이벌': '17_다크_재난', 
    'SF|미래|기갑|AI|메카닉물|인공지능': '18_SF_미래',
    '액션판타지|SF액션|SCHOOL_ACTION_FANTASY': '19_액션_하이브리드',
    '동양풍|WUXIA|무협': '20_동양풍_무협', 
    '서양풍|판타지|중세': '21_서양풍_판타지', 
    '아방|소심|순진|소녀|명랑|귀여운|털털녀|발랄|사랑스러운|명랑녀|햇살|애교|외유내강': '22_순수_발랄_청순', 
    # 23. 계략/까칠/나쁜남녀 보강 (캐릭터 성향)
    '계략|까칠|냉정|오만|나쁜|츤데레|소유욕|카리스마|짐승남|집착남|빌런|악녀|악역|절륜|오빠|여공남수|차도남|능글|도도|폭스|대형견남|폭군|남장여자|후계자|공식미남|바람둥이|직진남|직진녀|유혹남|유혹녀|존댓말남|다정남|다정녀|자상녀|무심남|무심녀|망나니|집착녀|집착물': '23_계략_까칠_나쁜남녀', 
    '군인': '24_군인_전쟁', 
    '음식|요리|먹방|식욕|요리사': '25_음식_요리',
    # 26. 초능력/영웅 보강
    '능력|초능력|영웅|신화|힐러|용사|마왕|네크로맨서|마검사|영지|이능력|이능력배틀물|악마|천사|신|히어로물|우월한|수인물|신데렐라|신령|조신남': '26_초능력_영웅', 
    '아이돌|연예인|가수': '27_아이돌_가수',
    '연상|연하|나이차': '28_연상연하',
    '결혼|선결혼|정략결혼': '29_결혼_선결혼',
    '가이드버스|헌터|레이드': '30_헌터물_레이드',
    '여주중심|여주판타지|걸크러쉬|군림녀|우월녀|공식미녀|얼빠녀': '31_여주_중심', 
    '남주중심|소년물|소년왕도물|먼치킨|힘숨찐': '32_남주_중심', 
    '4컷만화|웹툰|옴니버스': '33_단편_4컷', 
    '작가|소설원작|창작': '34_창작_원작',
    '공모전|최강자전|직행열차': '35_공모전_수상작',
    '정치|정치가|외교': '36_정치_외교',
    '의사|의학|메디컬': '37_의료_메디컬',
    '검|격투|무술': '38_격투_무술',
    '외국인|해외|혼혈': '39_해외_외국',
    # 40. 최종 잔여 (평가/메타/Niche - **극단적으로 소수만 남김**)
    '독자PICK|요즘핫한추천작|초강력추천|명작|작화가_좋은|궁극적으로|몰입되는|분량많은|선이 굵은|신선한|퀄리티 좋은|개성있는|과감한|기묘한|기발한|라이트노벨|레드스트링|레드아이스 스튜디오|레트로|로드무비|로판|만능|미친작화|소림|슈퍼스트링|유익한|음악|이야기중심|정통|지금추천작|지식정보|중년|프리퀄|환상적인|흑백|기타|권|동양|동정남|두근거리는|두뇌싸움|마계|마교|서양|서포터|선계|성좌|성직자|속도위반|여주지키기|움직임이 많은|자극적인|조력자|착각물|평범남|평범녀': '40_기타_분류불가', 
}

# 3. 그룹 할당 로직
def assign_group(genre):
    genre_clean = genre.strip()
    
    for keywords, group_name in MAPPING_RULES_V11.items(): 
        # re.IGNORECASE는 대소문자 구분을 없앱니다.
        if re.search(keywords, genre_clean, re.IGNORECASE):
            return group_name
            
    return '40_기타_분류불가'

filtered_df['cluster_name'] = filtered_df[target_column].apply(assign_group)

# 4. 결과 출력 및 요약
print("\n" + "="*60)
print("   ⭐ 최종 키워드 기반 장르 분류 결과 (40개 그룹, V11 최종 흡수)")
print("="*60)

summary_df = filtered_df.groupby('cluster_name').size().reset_index(name='count')
summary_df = summary_df.sort_values(by='cluster_name').reset_index(drop=True)

final_unclassified_count = 0

# 40번 그룹만 먼저 정확하게 출력 (남은 단어 확인 목적)
unclassified_data = filtered_df[filtered_df['cluster_name'] == '40_기타_분류불가']
if not unclassified_data.empty:
    final_unclassified_count = unclassified_data.shape[0]
    print(f"[{'40_기타_분류불가'}] (크기: {final_unclassified_count})")
    print(f"목록: {', '.join(unclassified_data[target_column].tolist())}")
    print("-" * 30)
else:
    print(f"[{'40_기타_분류불가'}] (크기: 0)")
    print("-" * 30)

# 나머지 그룹 출력
for index, row in summary_df.iterrows():
    group_name = row['cluster_name']
    group_count = row['count']
    
    if group_name == '40_기타_분류불가':
        continue # 40번은 이미 출력했으므로 건너뛰기
    
    items = filtered_df[filtered_df['cluster_name'] == group_name][target_column].tolist()
    
    print(f"[{group_name}] (크기: {group_count})")
    content = ', '.join(items)
    if len(content) > 150:
        print(f"목록: {content[:150]} ... (총 {group_count}개)")
    else:
        print(f"목록: {content}")
    print("-" * 30)

print(f"\n✅ '40_기타_분류불가' 그룹에 최종적으로 남은 단어 수: {final_unclassified_count}개")
print(f"✅ V11 규칙을 통해 195개의 잔여 단어를 최대한 흡수했습니다. 이 코드를 실행하시면 40번 그룹의 크기가 20개 내외로 줄어들 것입니다.")

# 5. 결과를 CSV 파일로 저장
output_file = 'final_40_grouped_genres_V11_FINAL.csv'
filtered_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n✅ 최종 결과가 '{output_file}' 파일로 저장되었습니다.")

분석 대상 단어 개수: 615

   ⭐ 최종 키워드 기반 장르 분류 결과 (40개 그룹, V11 최종 흡수)
[40_기타_분류불가] (크기: 106)
목록: 개성있는, 걸크러시, 고퀄리티, 과감한, 구원서사, 권, 권선징악, 기묘한, 기발한, 기사물, 기억상실, 긴장감 있는, 깽판, 낮져밤이, 농사, 뇌섹녀, 달달물, 당찬, 대장장이/생산직, 도사, 독, 독자PICK, 동거, 동거물, 동물, 동양, 동정남, 두근거리는, 두뇌싸움, 등골 오싹한, 라이벌, 라이벌/앙숙, 라이트노벨, 레드스트링, 레드아이스 스튜디오, 레트로, 로드무비, 로판, 마계, 마교, 만능, 명문세가, 명작, 모험, 모험물, 몰입되는, 몸정>맘정, 무당, 미친작화, 배틀, 뱀파이어, 분량많은, 블루스트링, 사연남, 사연녀, 사이다, 사이다남, 사이다녀, 사차원남, 상처남, 상처녀, 서양, 서포터, 선계, 성좌, 성직자, 소림, 속도위반, 순정남, 순정녀, 술법, 슈퍼스트링, 스포츠, 시한부, 아티팩트, 역하렘, 열혈, 예술, 요즘핫한추천작, 용병, 운명, 유익한, 음악, 의원, 이야기중심, 자극적인, 정통, 조력자, 중년, 지금추천작, 지식정보, 착각, 착각물, 평범남, 평범녀, 포식/흡수, 프리퀄, 하렘, 하렘물, 학사, 할리퀸, 현대, 현대물, 환상적인, 흑백, 히어로
------------------------------
[10_전문직_연예_스포츠] (크기: 25)
목록: BJ, PD/감독, 가수, 감독/스카우터, 기타 스포츠, 농구, 디자이너, 매니저, 방송, 배우, 법조인, 소방관, 스포츠 선수, 스포츠물, 아이돌, 야구, 야구선수, 연예계, 연예계물, 운동선수, 의사, 인플루언서, 전문직, 전문직물, 축구
------------------------------
[11_역사_시대극_궁중] (크기: 9)
목록: HISTORICAL, 가상시대물, 궁, 궁금하게 하는, 남궁, 대체역사, 시대물, 역사물, 왕족/귀족
------------------------------
[12_가족_육아

In [7]:
filtered_df

,raw_genre,cluster_name
0,2013 최강자전,35_공모전_수상작
1,2017 최강자전,35_공모전_수상작
2,2018 최강자전,35_공모전_수상작
3,2019 지상최대공모전,35_공모전_수상작
4,2020 지상최대공모전,35_공모전_수상작
...,...,...
610,히어로,40_기타_분류불가
611,힐러,26_초능력_영웅
612,힐링,4_드라마_일상_힐링
613,힐링물,4_드라마_일상_힐링


In [ ]:
tags = genre_df["raw_genre"].astype(str).str.strip()
tags = tags[tags != ""].unique().tolist()

In [36]:
tags_ = genre_df["raw_genre"].astype(str).str.strip()
need_trans = tags_[~tags_.str.contains("[가-힣]", regex=True)].unique().tolist()

In [37]:
translator = Translator()

def translate_en_to_ko(text: str, retries=3, delay=0.5) -> str:
    text = text.strip()
    if not text:
        return text
    for _ in range(retries):
        try:
            res = translator.translate(text, src="en", dest="ko")
            return res.text
        except Exception:
            time.sleep(delay)
    return text  # 실패 시 원문 유지

In [ ]:
cache = {}

for t in tqdm(need_trans):
    cache[t] = translate_en_to_ko(t)

100%|██████████| 24/24 [00:14<00:00,  1.64it/s]


In [41]:
cache["ACTION"] = "액션"
cache["ACTION_WUXIA"] = "무림_액션"
cache["AI"] = "인공지능"
cache["COMIC"] = "코믹"
cache["COMIC_EVERYDAY_LIFE"] = "일상_코믹"
cache["FANTASY_DRAMA"] = "판타지_드라마"
cache["SCHOOL_ACTION_FANTASY"] = "학원액션판타지"

In [42]:
cache

{'ACTION': '액션',
 'ACTION_WUXIA': '무림_액션',
 'AI': '인공지능',
 'BJ': 'BJ',
 'BL': 'BL',
 'COMIC': '코믹',
 'COMIC_EVERYDAY_LIFE': '일상_코믹',
 'DAILY': '일일',
 'DRAMA': '드라마',
 'FANTASY': '공상',
 'FANTASY_DRAMA': '판타지_드라마',
 'GL': 'GL',
 'HISTORICAL': '역사적인',
 'HORROR_THRILLER': '호러_스릴러',
 'PURE': '순수한',
 'ROMANCE': '로맨스',
 'ROMANCE_FANTASY': '로맨스_판타지',
 'SCHOOL_ACTION_FANTASY': '학원액션판타지',
 'SENSIBILITY': '감수성',
 'SF': 'SF',
 'SPORTS': '스포츠',
 'THRILL': '스릴',
 'TS': 'TS',
 'sf': 'SF'}

In [43]:
# 캐시를 CSV로 저장해두면 나중에 재사용 가능
pd.DataFrame({"raw_tag": list(cache.keys()), "ko_tag": list(cache.values())}) \
  .to_csv("tag_translation_cache.csv", index=False)

In [44]:
tags

['2013 최강자전',
 '2017 최강자전',
 '2018 최강자전',
 '2019 지상최대공모전',
 '2020 지상최대공모전',
 '2020 최강자전',
 '2021 지상최대공모전',
 '2021 최강자전',
 '2022 지상최대공모전',
 '2022 최강자전',
 '2023 지상최대공모전',
 '2024 연재직행열차',
 '2024 지상최대공모전',
 '2025 연재직행열차',
 '2025 지상최대공모전',
 '4차원',
 '4컷만화',
 'ACTION',
 'ACTION_WUXIA',
 'AI',
 'BJ',
 'BL',
 'COMIC',
 'COMIC_EVERYDAY_LIFE',
 'DAILY',
 'DRAMA',
 'FANTASY',
 'FANTASY_DRAMA',
 'GL',
 'HISTORICAL',
 'HORROR_THRILLER',
 'PD/감독',
 'PURE',
 'ROMANCE',
 'ROMANCE_FANTASY',
 'SCHOOL_ACTION_FANTASY',
 'SENSIBILITY',
 'SF',
 'SF액션',
 'SPORTS',
 'THRILL',
 'TS',
 'sf',
 '가난',
 '가벼운',
 '가상시대물',
 '가상현실',
 '가수',
 '가슴 먹먹한',
 '가이드버스',
 '가족',
 '가족물',
 '가족후회물',
 '감독/스카우터',
 '감동적인',
 '감성',
 '감성드라마',
 '감성적인',
 '감염',
 '갑을관계',
 '강공',
 '강한',
 '개그',
 '개그물',
 '개성있는',
 '객잔무협',
 '걸크러쉬',
 '걸크러시',
 '검',
 '검사',
 '게임',
 '게임판타지',
 '격투',
 '격투가',
 '격투기',
 '결혼',
 '결혼생활',
 '경영',
 '경영/기업',
 '경쟁구도',
 '경찰',
 '계략',
 '계략남',
 '계략녀',
 '계약관계',
 '계약연애',
 '계약연애/결혼',
 '고인물',
 '고자극드라마',
 '고자극로맨스',
 '고자극스릴러',
 '고퀄리티',
 '곤륜',
 

In [45]:
tag_map = cache  # 또는 CSV에서 다시 로드해서 dict로

for t in range(len(tags)):
    if tags[t] in cache.keys():
        tags[t] = cache[tags[t]]
        
tags

['2013 최강자전',
 '2017 최강자전',
 '2018 최강자전',
 '2019 지상최대공모전',
 '2020 지상최대공모전',
 '2020 최강자전',
 '2021 지상최대공모전',
 '2021 최강자전',
 '2022 지상최대공모전',
 '2022 최강자전',
 '2023 지상최대공모전',
 '2024 연재직행열차',
 '2024 지상최대공모전',
 '2025 연재직행열차',
 '2025 지상최대공모전',
 '4차원',
 '4컷만화',
 '액션',
 '무림_액션',
 '인공지능',
 'BJ',
 'BL',
 '코믹',
 '일상_코믹',
 '일일',
 '드라마',
 '공상',
 '판타지_드라마',
 'GL',
 '역사적인',
 '호러_스릴러',
 'PD/감독',
 '순수한',
 '로맨스',
 '로맨스_판타지',
 '학원액션판타지',
 '감수성',
 'SF',
 'SF액션',
 '스포츠',
 '스릴',
 'TS',
 'SF',
 '가난',
 '가벼운',
 '가상시대물',
 '가상현실',
 '가수',
 '가슴 먹먹한',
 '가이드버스',
 '가족',
 '가족물',
 '가족후회물',
 '감독/스카우터',
 '감동적인',
 '감성',
 '감성드라마',
 '감성적인',
 '감염',
 '갑을관계',
 '강공',
 '강한',
 '개그',
 '개그물',
 '개성있는',
 '객잔무협',
 '걸크러쉬',
 '걸크러시',
 '검',
 '검사',
 '게임',
 '게임판타지',
 '격투',
 '격투가',
 '격투기',
 '결혼',
 '결혼생활',
 '경영',
 '경영/기업',
 '경쟁구도',
 '경찰',
 '계략',
 '계략남',
 '계략녀',
 '계약관계',
 '계약연애',
 '계약연애/결혼',
 '고인물',
 '고자극드라마',
 '고자극로맨스',
 '고자극스릴러',
 '고퀄리티',
 '곤륜',
 '공감',
 '공감되는',
 '공감성수치',
 '공식미남',
 '공식미녀',
 '공포',
 '공포/스릴러',
 '과감한',
 '과거',
 '괴담',
 '구원서사',
 '군대물',

In [2]:
# %pip install rapidfuzz

In [46]:
from rapidfuzz import fuzz

n = len(tags)
sim = np.zeros((n, n))

for i in range(n):
    for j in range(i+1, n):
        score = fuzz.token_set_ratio(tags[i], tags[j]) / 100.0  # 0~1
        sim[i, j] = sim[j, i] = score

In [47]:
# %pip install scikit-learn

In [48]:
# 거리 = 1 - 유사도
dist = 1 - sim

clust = AgglomerativeClustering(
    metric="precomputed",
    linkage="average",
    distance_threshold=0.4,   # 임계값 (작을수록 더 촘촘하게 묶임)
    n_clusters=None,
)
labels = clust.fit_predict(dist)

In [49]:
cluster_df = pd.DataFrame({
    "raw_tag": tags,
    "cluster_id": labels,
})

# 각 클러스터별로 어떤 태그가 묶였는지 보기
clusters = (
    cluster_df
    .groupby("cluster_id")["raw_tag"]
    .apply(list)
    .reset_index()
)

clusters.head()

,cluster_id,raw_tag
0,0,"[연상연하, 연하, 연하공, 연하남]"
1,1,"[로맨스, 궁정로맨스, 궁중로맨스, 로맨스, 리얼로맨스, 청춘로맨스, 학원로맨스, ..."
2,2,"[사랑꾼, 짝사랑, 짝사랑남, 짝사랑녀, 첫사랑]"
3,3,"[게임판타지, 다크판타지, 여주판타지, 역사판타지, 판타지개그, 판타지물, 현대판타지]"
4,4,"[일상, 일상 판타지, 일상물]"


In [50]:
def pick_rep(tags):
    # 가장 짧은 태그를 대표로
    return sorted(tags, key=len)[0]

clusters["rep_tag"] = clusters["raw_tag"].apply(pick_rep)

# 다시 원시 태그별 매핑 테이블로 풀기
mapping = cluster_df.merge(clusters[["cluster_id","rep_tag"]], on="cluster_id")
mapping = mapping[["raw_tag","rep_tag"]]
clusters
# mapping.to_csv("tag_cluster_mapping.csv", index=False)

,cluster_id,raw_tag,rep_tag
0,0,"[연상연하, 연하, 연하공, 연하남]",연하
1,1,"[로맨스, 궁정로맨스, 궁중로맨스, 로맨스, 리얼로맨스, 청춘로맨스, 학원로맨스, ...",로맨스
2,2,"[사랑꾼, 짝사랑, 짝사랑남, 짝사랑녀, 첫사랑]",사랑꾼
3,3,"[게임판타지, 다크판타지, 여주판타지, 역사판타지, 판타지개그, 판타지물, 현대판타지]",판타지물
4,4,"[일상, 일상 판타지, 일상물]",일상
...,...,...,...
443,443,[능력있는],능력있는
444,444,[바보],바보
445,445,[경영/기업],경영/기업
446,446,[러브코미디],러브코미디


In [32]:
mapping

,raw_tag,rep_tag
0,2013 최강자전,최강자전
1,2017 최강자전,최강자전
2,2018 최강자전,최강자전
3,2019 지상최대공모전,2019 지상최대공모전
4,2020 지상최대공모전,2019 지상최대공모전
...,...,...
663,히어로,히어로
664,힐러,힐러
665,힐링,힐링
666,힐링물,힐링


In [31]:
# %pip install googletrans==4.0.0-rc1

In [ ]:
# %pip install sentence-transformers

  Using cached sentence_transformers-5.1.2-py3-none-any.whl.metadata (16 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached sentence_transformers-5.1.2-py3-none-any.whl (488 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ------------------- -------------------- 5.8/12.0 MB 29.3 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 30.0 MB/s  0:00:00
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   ---------------------------------------- 566.1/566.1 kB 19.0 MB/s  0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 38.9 MB/s  0:00:00
   ---------------------------------------- 0.0/111.0 MB ? eta -:--:--
   -- ---------

In [61]:
from sentence_transformers import SentenceTransformer

tags = genre_df["raw_genre"].tolist()

model = SentenceTransformer("jhgan/ko-sroberta-multitask")  # 예시 한국어 임베딩 모델
emb = model.encode(tags, batch_size=64, show_progress_bar=True)  # (N, d) 벡터

clust = AgglomerativeClustering(
    n_clusters=None,
    distance_threshold=1.5,   # 값 조절하면서 클러스터 개수 조정
    linkage="average",
    metric="euclidean",
)
labels = clust.fit_predict(emb)

cluster_df = pd.DataFrame({"raw_tag": tags, "cluster_id": labels})

Batches: 100%|██████████| 11/11 [00:03<00:00,  3.46it/s]


In [62]:
cluster_df

,raw_tag,cluster_id
0,2013 최강자전,505
1,2017 최강자전,546
2,2018 최강자전,596
3,2019 지상최대공모전,599
4,2020 지상최대공모전,342
...,...,...
663,히어로,18
664,힐러,8
665,힐링,1
666,힐링물,3


In [63]:
cluster_groups = (
    cluster_df  # raw_tag, cluster_id 가진 DataFrame
    .groupby("cluster_id")["raw_tag"]
    .apply(list)
    .reset_index()
)

# 태그가 3개 이상인 클러스터만 보기 (노이즈 제외)
big_clusters = cluster_groups[cluster_groups["raw_tag"].str.len() >= 3]

big_clusters.head()

,cluster_id,raw_tag
1,1,"[검, 공감, 무공, 뱀파이어, 법조인, 사이다, 살수, 소방관, 스릴러, 썸, 오..."
4,4,"[깽판, 소꿉친구, 츤데레, 츤데레공, 츤데레남, 츤데레수]"
